# Library

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os, re

# fold path

In [4]:
ods_fold = 'D:/data/big2/ODS'
col_fold = 'C:/Users/kbjung/Documents/GitHub/wabotech/car_big_data_2/analysis/ODSvsSTD'
issue_disclosure_fold = 'D:/data/big2/ODS/ODS_N_IS_ISSUE_DISCLOSURE'

In [5]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# 1\. 등록정보(ODS_CEG_CAR_MIG)

In [4]:
load_col_dtypes = {'VHMNO':'str', 'VHRNO':'str', 'BSPL_LEDO_CD':'int64', 'OWNR_SE':'str', 'VHCTY':'str', 'PURPS':'str', 'NOW_OWNR_NM':'str', 'NOW_MBERNO':'str', 'BSPL_ADRS':'str', 
 'OWNR_ADRS':'str', 'VIN':'str', 'FST_REG_DE':'int64', 'VHCL_YRIDNW':'int64', 'VHCL_MNFCT_DE':'str', 'EMIS_INSP_EFCT_DE':'float64', 'SRCMNNO':'str', 'EMIS_CRTCNO':'str', 
 'EMIS_GRD':'str', 'LEM_YN':'str', 'LEM_KND':'float64', 'LEM_STRCHG_YN':'str', 'BSPL_ZIP':'float64', 'VHRNO_NUM_INFO':'int64', 'VHCL_ERSR_YN':'str', 'FST_OWNR_NM':'str',
 'FST_MBERNO':'str', 'ACQS_DE':'str', 'ACQS_AMT':'float64', 'RMK':'str', 'CRTN_DT':'str', 'UPDT_DT':'str', 'USER_ID':'str'}

In [5]:
# about 9m 45s
# usecols 지정, dtype 지정 : 9m 27s
name = 'CEG_CAR_MIG'
file_name = f'ODS_{name}.csv'
df = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False, usecols=load_col_dtypes.keys(), dtype=load_col_dtypes)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29534047 entries, 0 to 29534046
Data columns (total 32 columns):
 #   Column             Dtype  
---  ------             -----  
 0   VHMNO              object 
 1   VHRNO              object 
 2   BSPL_LEDO_CD       int64  
 3   OWNR_SE            object 
 4   VHCTY              object 
 5   PURPS              object 
 6   NOW_OWNR_NM        object 
 7   NOW_MBERNO         object 
 8   BSPL_ADRS          object 
 9   OWNR_ADRS          object 
 10  VIN                object 
 11  FST_REG_DE         int64  
 12  VHCL_YRIDNW        int64  
 13  VHCL_MNFCT_DE      object 
 14  EMIS_INSP_EFCT_DE  float64
 15  SRCMNNO            object 
 16  EMIS_CRTCNO        object 
 17  EMIS_GRD           object 
 18  LEM_YN             object 
 19  LEM_KND            float64
 20  LEM_STRCHG_YN      object 
 21  BSPL_ZIP           float64
 22  VHRNO_NUM_INFO     int64  
 23  VHCL_ERSR_YN       object 
 24  FST_OWNR_NM        object 
 25  FST_MBERNO      

In [6]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
df.columns = [col_dict[x] for x in df.columns]
df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종', '용도', '미정', '현재회원번호',
       '미정', '미정', '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호',
       '배출가스인증번호', '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '미정',
       '차량등록번호숫자정보', '차량말소YN', '미정', '최초회원번호', '취득일자', '취득금액', '미정', '생성일시',
       '수정일시', '미정'],
      dtype='object')

## [로드] 정기&정밀(최신검사만)

In [31]:
# about 4m 33s
tin = pd.read_csv(os.path.join(ods_fold, '[ODS]정기&정밀검사(최신검사만)(2022.12.28).csv'), low_memory=False)
tin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22044208 entries, 0 to 22044207
Data columns (total 42 columns):
 #   Column     Dtype  
---  ------     -----  
 0   사용연료       object 
 1   주행거리       object 
 2   차명         object 
 3   차량번호       object 
 4   차대번호       object 
 5   차종         object 
 6   차량연식       int64  
 7   엔진형식       object 
 8   검사방법       object 
 9   검사일자       int64  
 10  검사종류       object 
 11  검사판정시각     object 
 12  검사판정       object 
 13  산소값        float64
 14  이산화탄소값     float64
 15  무부하매연판정1   object 
 16  무부하매연판정2   object 
 17  무부하매연판정3   object 
 18  무부하매연판정4   object 
 19  무부하매연판정5   object 
 20  무부하매연판정6   object 
 21  무부하매연허용치1  float64
 22  무부하매연허용치2  float64
 23  무부하매연허용치3  float64
 24  무부하매연허용치4  float64
 25  무부하매연허용치5  float64
 26  무부하매연허용치6  float64
 27  무부하매연측정치1  float64
 28  무부하매연측정치2  float64
 29  무부하매연측정치3  float64
 30  무부하매연측정치4  float64
 31  무부하매연측정치5  float64
 32  무부하매연측정치6  float64
 33  차량용도       object 
 34  최대출력허용치    float

In [8]:
tin.columns

Index(['사용연료', '주행거리', '차명', '차량번호', '차대번호', '차종', '차량연식', '엔진형식', '검사방법',
       '검사일자', '검사종류', '검사판정시각', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1',
       '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1',
       '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5',
       '무부하매연측정치6', '차량용도', '최대출력허용치', '최대출력측정값', '검사접수번호', '검사소명', '재검사기간',
       '정기정밀', '검사 검사소코드', '검사유효기간'],
      dtype='object')

## 차량등록번호

In [9]:
df.shape

(29534047, 32)

In [10]:
# 결측치 확인
df['차량등록번호'].isnull().sum()

0

In [11]:
p1 = re.compile('[가-힣]{2}[0-9]{1,2}[가-힣]{1}[0-9]{4}')
p2 = re.compile('[0-9]{2,3}[가-힣]{1}[0-9]{4}')
wrong_vhrno_list = []
for one in tqdm(df['차량등록번호']):
    if not p1.search(str(one)) and not p2.search(str(one)):
        wrong_vhrno_list.append(one)
len(wrong_vhrno_list)

100%|██████████| 29534047/29534047 [00:34<00:00, 852037.22it/s]


1

In [12]:
wrong_vhrno_list

['서울703**575']

In [17]:
df.loc[df['차량등록번호'] == '서울703**575', ['차대번호', '배출가스등급']]

,차대번호,배출가스등급
12072,03821215031,5


In [19]:
# about 7m 12s
total = df.merge(tin, on='차대번호', how='left')
total.shape

(29534047, 73)

In [20]:
total.loc[total['차대번호'] == '03821215031', '차량번호']

12072    NaN
Name: 차량번호, dtype: object

## 본거지 법정동 코드

In [13]:
df['본거지법정동코드'].isnull().sum()

0

In [12]:
wrong_localcode_list = []
for one in tqdm(df['본거지법정동코드']):
    if len(str(one)) != 10:
        wrong_localcode_list.append(one)
len(wrong_localcode_list)

100%|██████████| 29534047/29534047 [00:18<00:00, 1625711.43it/s]


0

In [13]:
p = re.compile('[0-9]{10}')
wrong_localcode_list = []
for one in tqdm(df['본거지법정동코드']):
    if not p.search(str(one)):
        wrong_localcode_list.append(one)
len(wrong_localcode_list)

100%|██████████| 29534047/29534047 [00:26<00:00, 1117989.31it/s]


0

## 배출가스검사유효일자

In [29]:
df['배출가스검사유효일자'].head()

0   19850225.00
1   19760706.00
2           NaN
3   20191220.00
4   19930227.00
Name: 배출가스검사유효일자, dtype: float64

In [30]:
df['배출가스검사유효일자'].isnull().sum()

635

In [31]:
null_idx = df[df['배출가스검사유효일자'].isnull() == True].index
len(null_idx)

635

In [32]:
p = re.compile('[0-9]{8}[.][0]')
wrong_gascheckdate_idx = []
for i, one in tqdm(enumerate(df['배출가스검사유효일자'].to_list())):
    if not p.search(str(one)):
        wrong_gascheckdate_idx.append(i)
len(wrong_gascheckdate_idx)

29534047it [00:32, 905707.47it/s]


637

In [33]:
set(wrong_gascheckdate_idx) - set(null_idx)

{4124, 15938}

In [34]:
df.loc[list(set(wrong_gascheckdate_idx) - set(null_idx)), '배출가스검사유효일자']

15938   1990604.00
4124    1990118.00
Name: 배출가스검사유효일자, dtype: float64

In [35]:
df.loc[list(set(wrong_gascheckdate_idx) - set(null_idx)), ['배출가스검사유효일자', '차량연식', '배출가스인증번호']]

,배출가스검사유효일자,차량연식,배출가스인증번호
15938,1990604.00,1983,NaN
4124,1990118.00,1979,NaN


In [36]:
df.loc[null_idx, ['차대번호', '배출가스검사유효일자', '차량연식', '배출가스인증번호']]

,차대번호,배출가스검사유효일자,차량연식,배출가스인증번호
2,10762211783,NaN,1976,NaN
8,10762211682,NaN,1976,NaN
10,10762212246,NaN,1976,NaN
47,10762213150,NaN,1976,NaN
52,03761307405,NaN,1976,NaN
...,...,...,...,...
8087297,KNAKU815BAA058010,NaN,2010,9MY-KM-14-22
8251095,KMFZCY7JAAU616346,NaN,2010,7MY-HD-14-68
20922089,KMHK4815GJU081024,NaN,2018,HMY-HD-14-31
26101534,KNANE81BBMS034872,NaN,2021,JMY-KM-14-29


In [62]:
df.loc[null_idx, ['차대번호', '배출가스검사유효일자', '차량연식', '배출가스인증번호']].iloc[50:60]

,차대번호,배출가스검사유효일자,차량연식,배출가스인증번호
609,KN70T108794,NaN,1970,NaN
612,03741224560,NaN,1974,NaN
627,03762203442,NaN,1976,NaN
630,03761204481,NaN,1976,NaN
658,10772229897,NaN,1977,NaN
669,10782210234,NaN,1978,NaN
699,10772217991,NaN,1977,NaN
706,03771214587,NaN,1977,NaN
748,KG70C206172000000,NaN,1976,NaN
749,03742123119,NaN,1974,NaN


In [72]:
total.loc[total['차대번호'] == '03742123119', '검사유효기간']

749   NaN
Name: 검사유효기간, dtype: float64

In [22]:
len('KMHK4815GJU081024')

17

In [23]:
df.loc[null_idx, ['차대번호', '배출가스검사유효일자', '차량연식', '배출가스인증번호']].isnull().sum()

차대번호            0
배출가스검사유효일자    635
차량연식            0
배출가스인증번호      540
dtype: int64

## [로드] 정기검사

In [24]:
# about 3m 34s
file_name = '[ODS]정기검사(2022.12.28).csv'
ins = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
ins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22006426 entries, 0 to 22006425
Data columns (total 39 columns):
 #   Column     Dtype  
---  ------     -----  
 0   사용연료       object 
 1   주행거리       object 
 2   차명         object 
 3   차량번호       object 
 4   차대번호       object 
 5   차종         object 
 6   차량연식       int64  
 7   엔진형식       object 
 8   검사방법       object 
 9   검사일자       int64  
 10  검사종류       object 
 11  검사판정시각     object 
 12  검사판정       object 
 13  산소값        float64
 14  이산화탄소값     float64
 15  무부하매연판정1   object 
 16  무부하매연판정2   object 
 17  무부하매연판정3   object 
 18  무부하매연판정4   object 
 19  무부하매연판정5   object 
 20  무부하매연판정6   object 
 21  무부하매연허용치1  float64
 22  무부하매연허용치2  float64
 23  무부하매연허용치3  float64
 24  무부하매연허용치4  float64
 25  무부하매연허용치5  float64
 26  무부하매연허용치6  float64
 27  무부하매연측정치1  float64
 28  무부하매연측정치2  float64
 29  무부하매연측정치3  float64
 30  무부하매연측정치4  float64
 31  무부하매연측정치5  float64
 32  무부하매연측정치6  float64
 33  차량용도       object 
 34  최대출력허용치    float

In [25]:
ins['검사일자'].head()

0    20190102
1    20190102
2    20190102
3    20190102
4    20190102
Name: 검사일자, dtype: int64

In [26]:
ins.loc[ins['차대번호'] == 'KMHK4815GJU081024', '검사일자']

18083379    20220124
Name: 검사일자, dtype: int64

## 배출가스 인증번호

In [27]:
df['배출가스인증번호'].isnull().sum()

3414724

## 차종

In [ ]:
df.isnull().sum()

In [9]:
df['차종'].unique()

array(['P', 'T', 'V', 'S', nan], dtype=object)

In [11]:
df.loc[df['차종'].isnull() == True, '제원관리번호'].head()

6257     20110100000000
7271     10000700110000
21694    10011400290003
26839    10000700500000
37972    10002200270001
Name: 제원관리번호, dtype: object

### [로드] 등록&제원정보

In [19]:
# about 9m 24s
file_name = '[ODS]등록정보&제원정보(2022.12.28).csv'
tdf = pd.read_csv(os.path.join(ods_fold, file_name), encoding='cp949', low_memory=False)
tdf.shape

(29534047, 49)

In [40]:
tdf.loc[tdf['배출가스인증번호'].isnull() == True, ['차대번호', '배출가스인증번호', '자동차형식', '엔진형식', '차명', '차량연식']].tail(10)

,차대번호,배출가스인증번호,자동차형식,엔진형식,차명,차량연식
29534031,KNMA4B2RMPP005538,NaN,T4W13-4D,H5H,SM6,2023
29534036,KMHJE811BPU100088,NaN,NX4H51BD-E8DF-G5A,G4FT,투싼 하이브리드(TUCSON HYBRID),2023
29534037,KMHJE811BPU100230,NaN,NX4H51BD-E8DO-G5A,G4FT,투싼 하이브리드(TUCSON HYBRID),2023
29534038,KMHJE811BPU100378,NaN,NX4H51BD-E9DO-G5A,G4FT,투싼 하이브리드(TUCSON HYBRID),2023
29534039,KMHJE811BPU100494,NaN,NX4H51BD-E8DF-G5A,G4FT,투싼 하이브리드(TUCSON HYBRID),2023
29534040,KMHJE811BPU100592,NaN,NX4H51BD-E8DF-G5A,G4FT,투싼 하이브리드(TUCSON HYBRID),2023
29534041,KMHL241JBPA060768,NaN,DNJBH-G7-G5A,G4NR,쏘나타 하이브리드 (SONATA HYBRID),2023
29534042,KMHL241JBPA060809,NaN,DNJBH-G7-G5A,G4NR,쏘나타 하이브리드 (SONATA HYBRID),2023
29534043,KMHL241JBPA060817,NaN,DNJBH-G7-G5A,G4NR,쏘나타 하이브리드 (SONATA HYBRID),2023
29534046,KMHL3412BPA299998,NaN,DN3BF-VS9-G3A,G4FP,쏘나타(SONATA),2023


In [33]:
tdf.loc[tdf['자동차형식'] == 'FX-2', ['배출가스인증번호', '차량연식', '차명']].value_counts()

배출가스인증번호   차량연식  차명   
LMY-HD-08  1988  엑셀       25
           1987  엑셀        6
KMY-HD-02  1987  엑셀        1
           1988  엑셀        1
           1989  엑셀        1
KMY-HD-04  1992  EXCEL     1
LMY-HD-08  1989  엑셀        1
dtype: int64

In [42]:
tdf.loc[tdf['엔진형식'] == 'G4FP', ['배출가스인증번호', '차량연식', '차명']].value_counts()

배출가스인증번호      차량연식  차명                      
KMY-KM-14-81  2020  K5                          24414
LMY-HD-14-62  2021  투싼(TUCSON)                  23281
MMY-KM-14-36  2022  스포티지                        21174
LMY-HD-14-62  2022  투싼(TUCSON)                  20925
KMY-KM-14-81  2021  K5                          15056
JMY-KM-14-57  2022  스포티지                        12525
KMY-HD-14-56  2021  쏘나타(SONATA)                 10923
              2022  쏘나타(SONATA)                  9652
KMY-KM-14-81  2022  K5                           9458
KMY-HD-14-56  2020  쏘나타(SONATA)                  9007
NMY-KM-14-11  2023  셀토스                          8869
HMY-HD-14-31  2021  코나(KONA)                     7682
LMY-HD-14-76  2022  코나(KONA)                     2485
              2023  코나(KONA)                     2351
LMY-HD-14-33  2021  아반떼 N 라인(AVANTE N Line)      1613
MMY-KM-14-36  2023  스포티지                         1350
JMY-HD-14-48  2021  투싼(TUCSON)                   1070
LMY-HD-14-62  2023  투싼(TUCSON)       

## 차종

In [44]:
tdf['차종_등록정보'].isnull().sum()

14

In [45]:
tdf['차종_등록정보'].unique()

array(['승용', '화물', '승합', '특수', nan], dtype=object)

In [48]:
tdf.loc[tdf['차종_등록정보'].isnull() == True, ['차명', '차종_등록정보', '차종분류']]

,차명,차종_등록정보,차종분류
6257,유니목1200,NaN,특수용도형-특수용도형
7271,타이탄디젤14,NaN,일반형-카고
21694,봉고킹캡,NaN,일반형-카고
26839,슈퍼타이탄디젤14척(2.5톤),NaN,일반형-카고
37972,로얄프린스1.5,NaN,일반
38473,그레이스살롱,NaN,일반
43509,봉고킹캡,NaN,일반형-카고
47889,포니2-1400픽업컨버터블탑,NaN,일반형-픽업
60693,베스타3밴,NaN,밴
62028,점보타이탄2.5톤더블캡,NaN,일반형-카고


## 차량 제작 일자

In [49]:
tdf['차량제작일자'].isnull().sum()

424546

In [52]:
tdf['차량제작일자'].describe()

count   29109501.00
mean    20133035.32
std        62241.75
min        11212.00
25%     20090924.00
50%     20140829.00
75%     20180908.00
max     20221227.00
Name: 차량제작일자, dtype: float64

In [53]:
tdf.loc[tdf['차량제작일자'].isnull() == True, ['차량제작일자', '차량연식']].head()

,차량제작일자,차량연식
0,NaN,1976
1,NaN,1968
2,NaN,1976
3,NaN,1971
4,NaN,1986


In [55]:
tdf['차량제작일자'] = tdf['차량제작일자'].astype(str)

In [58]:
tdf['차량제작일자'].tail()

29534042    20221222.0
29534043    20221222.0
29534044    20221209.0
29534045    20221216.0
29534046    20221219.0
Name: 차량제작일자, dtype: object

In [59]:
tdf.loc[tdf['차량제작일자'].str.len() != 10].shape

(424560, 49)

In [62]:
tdf.loc[tdf['차량제작일자'].str.len() != 10, '차량제작일자'].tail()

21685676          nan
21712859          nan
22655861          nan
23415596          nan
28698925    2020101.0
Name: 차량제작일자, dtype: object

In [65]:
nan_idx = []
for i, one in tqdm(enumerate(tdf['차량제작일자'].to_list())):
    if str(one) == 'nan':
        nan_idx.append(i)
len(nan_idx)

29534047it [00:14, 2067135.81it/s]


424546

In [66]:
error_idx = []
for i, one in tqdm(enumerate(tdf['차량제작일자'].to_list())):
    if len(str(one)) != 10:
        error_idx.append(i)
len(error_idx)

29534047it [00:16, 1765138.99it/s]


424560

In [67]:
len(set(error_idx) - set(nan_idx))

14

In [70]:
tdf.loc[list(set(error_idx) - set(nan_idx)), ['차량제작일자', '차량연식']]

,차량제작일자,차량연식
17678839,2160221.0,2016
1492297,11212.0,2002
3291562,50226.0,2005
858730,7770707.0,1999
5554988,70814.0,2008
28698925,2020101.0,2022
12484270,2130128.0,2013
1233232,2210413.0,2001
201553,9910501.0,1991
453521,9950407.0,1995


In [71]:
tdf.loc[nan_idx, ['차량제작일자', '차량연식']].head()

,차량제작일자,차량연식
0,nan,1976
1,nan,1968
2,nan,1976
3,nan,1971
4,nan,1986


In [12]:
df.loc[df['차량제작일자'] == '1996 308', ['차량제작일자', '차량연식']]

,차량제작일자,차량연식
530017,1996 308,1996


## 차량 연식

In [72]:
df['차량연식'].isnull().sum()

0

In [73]:
p = re.compile('[0-9]')
count = 0
if one in df['차량연식']:
    if not p.search(str(one)):
        count += 1
count

0

## 차량 말소 YN

In [74]:
df['차량말소YN'].isnull().sum()

0

In [75]:
df['차량말소YN'].value_counts(dropna=True)

N    25693532
Y     3840515
Name: 차량말소YN, dtype: int64

## 차대번호

In [76]:
df['차대번호'].isnull().sum()

0

In [10]:
vin_temp = df['차대번호'].str.len()
set(vin_temp.unique())

{7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18}

In [12]:
count = 0
for one in set(vin_temp.unique()):
    temp = df.loc[df['차대번호'].str.len() == one, '차대번호']
    count += temp.shape[0]
    print(f'차대번호 길이 : {one}')
    print(temp.shape)
    display(temp.head())
    print()
count

차대번호 길이 : 7
(1,)


4476    2EJ1684
Name: 차대번호, dtype: object


차대번호 길이 : 8
(4,)


10312     14072632
19219     S2159974
64281     TX000402
666792    KPO42337
Name: 차대번호, dtype: object


차대번호 길이 : 9
(29,)


...


차대번호 길이 : 17
(29497619,)


3     PN71C018511000000
4     KMHLA11FPGU060639
9     SL67C000000027766
44    000000KG701300199
80    000000SL44C003630
Name: 차대번호, dtype: object


차대번호 길이 : 18
(1,)


613675    KMFLA19JPTC1098071
Name: 차대번호, dtype: object

29534047

In [13]:
len_17_wrong_text_idx = []
p = re.compile('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]')
for i, one in enumerate(df['차대번호']):
    if len(str(one)) == 17 and p.search(str(one)):
        len_17_wrong_text_idx.append(i)
len(len_17_wrong_text_idx)

0

In [ ]:
df.loc[len_17_wrong_text_idx, '차대번호'].head()

## 제원관리번호

In [14]:
df['제원관리번호'].isnull().sum()

0

In [15]:
df['제원관리번호'].head()

0    10004800000000
1    30220100010946
2    10004800000000
3    30321400001063
4    30381300100968
Name: 제원관리번호, dtype: object

In [16]:
temp = df['제원관리번호'].str.len()
temp.unique()

array([14, 17], dtype=int64)

In [17]:
df.loc[df['제원관리번호'].str.len() == 14, '제원관리번호'].shape

(1998450,)

In [18]:
df.loc[df['제원관리번호'].str.len() == 14, '제원관리번호'].head()

0    10004800000000
1    30220100010946
2    10004800000000
3    30321400001063
4    30381300100968
Name: 제원관리번호, dtype: object

In [82]:
df.loc[df['제원관리번호'].str.len() == 17, '제원관리번호'].shape

(27535597,)

In [83]:
df.loc[df['제원관리번호'].str.len() == 17, '제원관리번호'].head()

33     00034313000053303
64     00034717000033303
107    00032709000020003
178    00034113000441104
194    00034314000023303
Name: 제원관리번호, dtype: object

In [84]:
error_idx = []
p = re.compile('[가-힣]')
for i, one in enumerate(df['제원관리번호']):
    if p.search(str(one)):
        error_idx.append(i)
len(error_idx)

0

## 배출가스 등급

In [9]:
df['배출가스등급'].isnull().sum()

32

In [10]:
df.loc[df['배출가스등급'].isnull() == True, ['차종', '차량연식']].head()

,차종,차량연식
6371951,P,2009
6388290,P,2009
6423602,P,2009
6433596,P,2009
6443163,P,2009


In [11]:
df.loc[df['배출가스등급'].isnull() == True, ['차종', '차량연식']].tail()

,차종,차량연식
7233033,P,2009
7286470,P,2009
7308340,P,2009
7359054,P,2009
7419773,P,2009


In [14]:
tdf.loc[tdf['배출가스등급'].isnull() == True, ['제원관리번호', '차종_등록정보', '차량연식', '연료']].head()

,제원관리번호,차종_등록정보,차량연식,연료
6371951,A0110003200611408,승용,2009,휘발유
6388290,A0110003200611408,승용,2009,휘발유
6423602,A0110003200611408,승용,2009,휘발유
6433596,A0110003200611408,승용,2009,휘발유
6443163,A0110003200611408,승용,2009,휘발유


In [17]:
df.loc[(df['제원관리번호'] == 'A0110003200611408') & (df['배출가스등급'].isnull() == True), '배출가스등급']

6371951    NaN
6388290    NaN
6423602    NaN
6433596    NaN
6443163    NaN
6490573    NaN
6511542    NaN
6581976    NaN
6708567    NaN
6751967    NaN
6820551    NaN
6899089    NaN
6908881    NaN
6980361    NaN
7180758    NaN
7209206    NaN
7220856    NaN
7233033    NaN
7359054    NaN
Name: 배출가스등급, dtype: object

# 2\. [로드] 제원정보(ODS_CEG_CAR_SRC_MIG)

In [6]:
# about 5s
name = 'CEG_CAR_SRC_MIG'
file_name = f'ODS_{name}.csv'
src = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
src.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545374 entries, 0 to 545373
Data columns (total 28 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   SRCMNNO      545374 non-null  object 
 1   VHCNM        545272 non-null  object 
 2   VHCTY        545370 non-null  object 
 3   VHCTY_CL     531367 non-null  object 
 4   VHCTY_TY     531332 non-null  object 
 5   CAR_FRM      516295 non-null  object 
 6   MKR_NM       529017 non-null  object 
 7   FUEL         503254 non-null  object 
 8   EGINTY       494157 non-null  object 
 9   EMPV_WGHT    0 non-null       float64
 10  TOT_WGHT     541702 non-null  float64
 11  CRYNG_WGHT   446181 non-null  float64
 12  EGINPWR      471948 non-null  float64
 13  DSPLVL       510521 non-null  float64
 14  TKCAR_NMPR   545313 non-null  float64
 15  VHCL_LT      514786 non-null  float64
 16  VHCL_WH      514799 non-null  float64
 17  VHCL_HG      514732 non-null  float64
 18  PLOR_NM      540392 non-

In [7]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
src.columns = [col_dict[x] for x in src.columns]
src.columns

Index(['제원관리번호', '차명', '차종', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료',
       '엔진형식', '공차중량', '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비',
       '차량높이', '원산지명', '미정', '삭제YN', '승인일자', '구동형식', '변속기종류', '미정', '생성일시',
       '수정일시', '미정'],
      dtype='object')

## 총 중량

In [8]:
src['총중량'].describe()

count   541702.00
mean     10034.74
std      12666.19
min          0.00
25%       1795.00
50%       3135.00
75%      15055.00
max     936130.00
Name: 총중량, dtype: float64

In [9]:
src.loc[src['총중량'].isnull() == True, ['자동차형식', '엔진형식']].head(20)

,자동차형식,엔진형식
78,NaN,NaN
108,0,0
114,0,0
122,0,NaN
124,0,NaN
126,0,NaN
127,0,NaN
282,0,NaN
903,0,NaN
1081,BH-G38T-A2,G6DA


In [10]:
src.loc[src['자동차형식'] == 'BH-G38T-A2', ['자동차형식', '엔진형식','총중량']]

,자동차형식,엔진형식,총중량
1081,BH-G38T-A2,G6DA,NaN
152612,BH-G38T-A2,G6DA,2135.00
207047,BH-G38T-A2,G6DA,2135.00
209767,BH-G38T-A2,G6DA,2135.00
344750,BH-G38T-A2,G6DA,2135.00
350148,BH-G38T-A2,G6DA,2135.00
369175,BH-G38T-A2,G6DA,2135.00


## 차종 유형(소분류)

In [28]:
src['차종유형(소분류)'].unique()

array(['L', 'M', 'S', 'C', nan], dtype=object)

In [19]:
temp = tdf.loc[tdf['차종유형(소분류)'].isnull() == True, '자동차형식']
for one in temp:
    tp = tdf.loc[tdf['자동차형식'] == one, ['자동차형식', '차종유형(소분류)']]
    if tp.shape[0] != 0:
        print(tp)

                자동차형식 차종유형(소분류)
2547  HDH-25-LST 1978       NaN
       자동차형식 차종유형(소분류)
3555    1973       NaN
699001  1973        소형
               자동차형식 차종유형(소분류)
104415  ALSA1830L/SE       NaN
104621  ALSA1830L/SE       NaN
105668  ALSA1830L/SE       NaN
113391  ALSA1830L/SE       NaN
               자동차형식 차종유형(소분류)
104415  ALSA1830L/SE       NaN
104621  ALSA1830L/SE       NaN
105668  ALSA1830L/SE       NaN
113391  ALSA1830L/SE       NaN
               자동차형식 차종유형(소분류)
104415  ALSA1830L/SE       NaN
104621  ALSA1830L/SE       NaN
105668  ALSA1830L/SE       NaN
113391  ALSA1830L/SE       NaN
              자동차형식 차종유형(소분류)
55996   HEC-F2-2500        대형
60893   HEC-F2-2500        대형
88667   HEC-F2-2500        대형
88720   HEC-F2-2500        대형
90012   HEC-F2-2500        대형
...             ...       ...
569846  HEC-F2-2500        대형
570803  HEC-F2-2500        대형
571814  HEC-F2-2500        대형
614951  HEC-F2-2500        대형
710290  HEC-F2-2500        대형

[183 rows x 2 columns]
               자동차

In [20]:
temp = tdf.loc[tdf['차종유형(소분류)'].isnull() == True, '엔진형식']
for one in temp:
    tp = tdf.loc[tdf['엔진형식'] == one, ['엔진형식', '차종유형(소분류)']]
    if tp.shape[0] != 0:
        print(tp)

## 차종분류

In [34]:
src['차종분류'].unique()

array(['특수용도형-특수용도형', '특수', '특수용도형-피견인', '일반', '다목적', '밴', '일반형-픽업',
       '일반형-카고', '기타', '견인', '특수작업형', '특수용도형-탱크로리', '덤프', '구난', '승용겸화물',
       '특수용도형', nan], dtype=object)

In [35]:
src['차종분류'].isnull().sum()

14007

In [47]:
src.loc[src['차종분류'].isnull() == True, ['제원관리번호', '차종분류', '차종', '차명']].tail()

,제원관리번호,차종분류,차종,차명
545114,A0350008620033022,NaN,T,구쎈
545115,A0350008620043022,NaN,T,구쎈
545116,A0350008620053022,NaN,T,구쎈
545117,A0350008620063022,NaN,T,구쎈
545118,A0350008620073022,NaN,T,구쎈


In [102]:
src['차종'].unique()

array(['T', 'V', 'P', 'S', nan], dtype=object)

In [86]:
df['차종분류'].unique()

array(['일반', '일반형-카고', '특수용도형-특수용도형', '특수용도형-피견인', '덤프', '구난', '특수작업형',
       '기타', '다목적', '견인', '밴', '일반형-픽업', '승용겸화물', '특수용도형-탱크로리', '특수',
       '특수용도형', nan], dtype=object)

In [87]:
df['차종분류'].isnull().sum()

21

In [41]:
df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종', '용도', '미정', '현재회원번호',
       '미정', '미정', '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호',
       '배출가스인증번호', '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '미정',
       '차량등록번호숫자정보', '차량말소YN', '미정', '최초회원번호', '취득일자', '취득금액', '미정', '생성일시',
       '수정일시', '미정'],
      dtype='object')

In [52]:
df.loc[df['제원관리번호'] == 'A0350008620073022', '차종']

Series([], Name: 차종, dtype: object)

## 차종

In [112]:
src['차종'].unique()

array(['T', 'V', 'P', 'S', nan], dtype=object)

In [113]:
src['차종'].isnull().sum()

4

In [114]:
src.loc[src['차종'].isnull() == True, ['제원관리번호' ,'차종분류']]

,제원관리번호,차종분류
67558,30110100002329,NaN
171109,30210100000056,NaN
275351,30220100000001,NaN
318490,30110100002328,NaN


In [122]:
df['차종'].isnull().sum()

14

In [128]:
for one in src.loc[src['차종'].isnull() == True, ['제원관리번호' ,'차종분류']]['제원관리번호'].values:
    display(df.loc[df['제원관리번호'] == one, ['제원관리번호', '차종']])

,제원관리번호,차종


,제원관리번호,차종


,제원관리번호,차종


,제원관리번호,차종


In [130]:
for one in src.loc[src['차종'].isnull() == True, ['제원관리번호' ,'차종분류']]['제원관리번호'].values:
    display(tdf.loc[tdf['제원관리번호'] == one, ['제원관리번호', '차종_등록정보', '차종_제원정보']])

,제원관리번호,차종_등록정보,차종_제원정보


,제원관리번호,차종_등록정보,차종_제원정보


,제원관리번호,차종_등록정보,차종_제원정보


,제원관리번호,차종_등록정보,차종_제원정보


## 차명

In [125]:
src['차명'].isnull().sum()

102

In [136]:
src.loc[src['차명'].isnull() == True, ['자동차형식', '엔진형식', '차명']].head(30)

,자동차형식,엔진형식,차명
SRCMNNO,,,
30432000000014,NaN,NaN,NaN
30451600000004,NaN,NaN,NaN
30451600000007,NaN,10831268192,NaN
30451600000008,NaN,XA014091,NaN
30220100001103,NaN,NaN,NaN
30481800000001,NaN,G15CF,NaN
30460300000009,NaN,NaN,NaN
30462500000000,NaN,NaN,NaN
30110100002429,NaN,01801300465,NaN


In [134]:
df.loc[df['자동차형식'] == '104-941', ['자동차형식', '엔진형식', '차명']]

,자동차형식,엔진형식,차명
517100,104-941,104.941,벤즈C280
1878718,104-941,112 920,벤츠C280


In [137]:
df.loc[df['엔진형식'] == 'G15CF', ['자동차형식', '엔진형식', '차명']]

,자동차형식,엔진형식,차명
1171,TM19,G15CF,르망
1355,TM19,G15CF,르망
1972,TM19,G15CF,르망
3418,TM19,G15CF,르망
4413,TA19,G15CF,르망오토
...,...,...,...
1009310,TF19,G15CF,대우 르망
1087713,TM19,G15CF,르망
2082125,TF19E,G15CF,대우 르망
2310856,TF19,G15CF,LEMANS


In [132]:
df['차명'].isnull().sum()

23

## 제작사 명

In [11]:
src['제작사명'].isnull().sum()

16357

In [12]:
list(src['제작사명'].unique())

['(주) 두성특장차',
 '자기인증면제차량',
 'SOFA차량',
 '한국토요타자동차(주)',
 '현대자동차(주)',
 '닛산',
 'BMW AG',
 '자일대우상용차 주식회사',
 '(주)참존임포트',
 '한국특장차(주)',
 '스카니아코리아그룹(주)',
 '타타대우상용차(주)',
 '주식회사 천하',
 '아우라(AURA)',
 '정우정공 (주)',
 '원모터스(One motors)',
 '기아자동차(주)',
 '미니앤컴팩(mini&compact)',
 '와우모터스',
 '홍성일',
 '에스알(S.R)모터스',
 '이명환',
 '세하(SEHA)',
 '(주)인피니코리아',
 '오토포럼',
 '스타자동차매매상사',
 '최영훈',
 '(주)노바모터스',
 '조인수입차',
 '(주)신광테크놀러지',
 '쌍용자동차(주)',
 '한국상용트럭(주)',
 '(주)한성특장',
 '아우디폭스바겐코리아(주)',
 '(주)한국쓰리축',
 '(주)새한에어서스펜션',
 '모터프로',
 '한신특장',
 '명성정공',
 '뉴클락모터스',
 '한남상사',
 '더블유모터스컴퍼니',
 '현대',
 '한국특장기술(주)',
 nan,
 '(주)수산특장',
 '대우자동차(주)',
 '(주)스타텍',
 '케이에이치특장(주)',
 '(유)동양특장차',
 '(주)함코',
 '(유)삼능기계',
 '동우특장차(주)',
 '(주)알텍',
 '미래특장차(주)',
 '금양기전',
 '삼성상용차(주)',
 '지엠코리아(주)',
 '아시아자동차공업(주)',
 '**기타**',
 '(주)진도',
 'TOYOTA',
 'MAZDA',
 '건민특장 주식회사',
 '디와이(주) 익산공장',
 '(주)한국그린피아',
 '(주)포멕특장',
 '(주)한중특장',
 '대우중공업(주)',
 '대양중공업(주)',
 '이텍산업 주식회사',
 '메리트특장차주식회사',
 '태산냉동 주식회사',
 '(주)동해기계항공',
 '(주)유로카라반',
 '(주)한국토미',
 '도쿄모터스',
 '주식회사 에타',
 '엔에스티네트웍스(주)',
 '

In [13]:
src.loc[src['제작사명'].isnull() == True, ['차명', '자동차형식', '엔진형식', '제작사명']].head(10)

,차명,자동차형식,엔진형식,제작사명
357,벤츠300E,124 030,103 983,NaN
358,벤즈200,NaN,10292010047640,NaN
359,벤츠300E,124 030,103 983,NaN
361,벤츠S320,140 033,104 994,NaN
362,볼보,740GL,B200E,NaN
363,머큐리세이블,MSV3-27C,MSV3,NaN
364,아우디A62.0E,4A,ABK,NaN
365,BMW,318I,184S1,NaN
367,랜드크루사,NaN,3F0117036,NaN
376,BMW318i,318IA,184S1,NaN


In [14]:
src.loc[src['제작사명'].isnull() == True, ['차명', '자동차형식', '엔진형식', '제작사명']].shape

(16357, 4)

In [15]:
src.loc[src['제작사명'].isnull() == True, ['차명', '자동차형식', '엔진형식', '제작사명']].isnull().sum()

차명          57
자동차형식     5331
엔진형식      4164
제작사명     16357
dtype: int64

In [16]:
src.loc[src['차명'] == '벤츠300E', '제작사명'].unique()

array([nan, '-', '**기타**', '다임러크라이슬러(주)', 'BENZ', '메르세데스벤츠코리아(주)',
       '자기인증면제차량', '(주) 송림모터스', '유럽자동차판매(주)', '선인관리 (주)'], dtype=object)

In [17]:
src.loc[src['자동차형식'] == '740GL', '제작사명'].unique()

array([nan, '볼보', '**기타**'], dtype=object)

In [20]:
src.loc[src['자동차형식'] == '4A', ['제작사명', '차명']]

,제작사명,차명
364,NaN,아우디A62.0E
7008,**기타**,아우디
16637,**기타**,아우디A62.8QUATTRO
17439,**기타**,아우디90
23171,아우디,아우디A62.4
...,...,...
344340,-,아우디A62.6E
386618,NaN,도요다코로나
396152,NaN,AUDI2.8CABRIOLET
403299,NaN,아우디A62.8E


In [21]:
src.loc[src['자동차형식'] == '4A', '제작사명'].unique()

array([nan, '**기타**', '아우디', '자기인증면제차량', '극동유화(주) 고진모터 임포츠',
       '(주) 아이엠씨트레이딩', '-'], dtype=object)

## 제원관리번호

In [21]:
src['제원관리번호'].isnull().sum()

0

In [22]:
src.loc[src['제원관리번호'].str.len() == 14, '제원관리번호'].shape

(73894,)

In [23]:
src.loc[src['제원관리번호'].str.len() == 14, '제원관리번호'].head()

95     20446900000000
96     20447300000000
356    30111100000069
357    30111100000072
358    30111100000073
Name: 제원관리번호, dtype: object

In [24]:
src.loc[src['제원관리번호'].str.len() == 17, '제원관리번호'].shape

(471456,)

In [18]:
error_idx = []
p = re.compile('[가-힣]')
for i, one in enumerate(src['제원관리번호']):
    if p.search(str(one)):
        error_idx.append(i)
len(error_idx)

0

## 자동차 형식

In [11]:
src['자동차형식'].isnull().sum()

29079

In [12]:
tdf.loc[(tdf['자동차형식'].isnull() == True) & (tdf['차명'] == 'K5'), ['제원관리번호', '자동차형식', '차명', '차량연식']]

,제원관리번호,자동차형식,차명,차량연식
29528902,99932714016771222,NaN,K5,2022


In [13]:
src.loc[src['차명'] == 'K5', ['자동차형식', '제원관리번호']].value_counts()

자동차형식       제원관리번호           
DL32BA-A7   A0110006900001219    1
JFS6BC-LS6  A0110005802041216    1
JFS6BA-TE   A0110005800671215    1
JFS6BB-GE   A0110005801451215    1
JFS6BB-L    A0110005801421215    1
                                ..
JF1BH-L6    A0110005803321219    1
JF1BH-L7    A0110005803331219    1
JF1BH-L8    A0110005803341219    1
JF1BH-P6    A0110005803381219    1
TF49BG-SE   A0110004701501214    1
Length: 878, dtype: int64

In [14]:
tdf.loc[tdf['제원관리번호'] == 'A0110006900001219', '차량연식']

24376836    2020
24378878    2020
24422716    2020
24431872    2020
24440310    2020
            ... 
25922052    2020
25922053    2020
25947527    2020
26018539    2020
26059602    2020
Name: 차량연식, Length: 350, dtype: int64

In [15]:
tdf.loc[(tdf['자동차형식'].isnull() == True) & (tdf['차명'] == 'K5'), ['제원관리번호', '자동차형식', '차명', '차량연식']]

,제원관리번호,자동차형식,차명,차량연식
29528902,99932714016771222,NaN,K5,2022


In [16]:
tdf.loc[tdf['차명'] == 'K5', ['자동차형식', '차량연식']].value_counts()

자동차형식           차량연식
TF-1N6AB-1      2011    56353
TF-5T5AB-1      2011    26452
TF-1B1-S        2012    21709
TF-8B1J-L93     2013    20208
TF-9B1G-92      2013    20078
                        ...  
DL3ABC-LV8-G3A  2022        1
TF-5T5AB-1      2010        1
TF-1N6MB-1      2011        1
JFS6AC-L6       2017        1
JFS6AF-L6       2018        1
Length: 478, dtype: int64

In [17]:
tdf.loc[(tdf['차명'] == 'K5') & (tdf['차량연식'] == 2022), ['자동차형식', '차량연식']].value_counts()

자동차형식            차량연식
JF6BL-X-G5A      2022    3500
DL3ABC-G7-G3A    2022    3265
DL3ABC-S8-G3A    2022    3134
DL32BC-C9-G3A    2022    2679
DL3BBC-G7-G5A    2022    2634
DL32BC-D9-G3A    2022    2314
DL3BBC-F6-G5A    2022    1971
DL3ABC-G8-G3A    2022    1772
DL3ABC-S7-G3A    2022    1770
DL3ABC-T8-G3A    2022    1477
DL32BC-D8-G3A    2022    1191
DL3ABC-L7-G3A    2022    1018
DL32BC-B9-G3A    2022     988
DL32BC-DV9-G3A   2022     892
DL3ABC-TV8-G3A   2022     628
DL32BC-C8-G3A    2022     496
DL32BC-B8-G3A    2022     493
DL3BBC-A7-G5A    2022     312
DL3BBC-B7-G5A    2022     294
DL3ABC-SV8-G3A   2022     286
DL3BBC-C8-G5A    2022     276
DL32BC-CV9-G3A   2022     223
DL3BBC-CV8-G5A   2022     137
DL32BC-A7-G3A    2022     115
DL3ABC-L8-G3A    2022      80
DL3ABC-SV7-G3A   2022      75
DL3ABC-GV7-G3A   2022      64
DL3ABC-GV8-G3A   2022      49
DL32BC-BV9-G3A   2022      37
DL32BC-A9-G3A    2022      34
DL3BBC-E7-G5A    2022      20
DL3BBC-D7-G5A    2022      15
DL3BBC-F28-G5A   2

## 연료

In [15]:
src['연료'].value_counts(dropna=False)

DS     269767
GS     213823
NaN     42120
CN       7192
LP       4804
GH       3780
EV       1720
OT       1583
DH        319
CH        105
HD         78
LN         23
LH         19
KS         18
AC         13
SH         10
Name: 연료, dtype: int64

In [16]:
use_dict = {'DS':'경유', 'GS':'휘발유', 'CN':'CNG(압축천연가스)', 'GH':'휘발유 하이브리드', 'LP':'LPG(액화석유가스)', 'OT':'기타연료', 'AC':'알코올', 'EV':'전기', 'SH':'태양열', 'CH':'CNG 하이브리드',
       'KS':'등유', 'DH':'경유 하이브리드', 'LH':'LPG 하이브리드', 'HD':'수소', 'LN':'LNG(액화천연가스)', 'PH':'플러그인 하이브리드', 'X':'무동력'}
src['연료'] = src['연료'].map(use_dict)
src['연료'].unique()

array([nan, '경유', '휘발유', 'CNG(압축천연가스)', '휘발유 하이브리드', 'LPG(액화석유가스)',
       '기타연료', '알코올', '전기', '태양열', 'CNG 하이브리드', '등유', '경유 하이브리드',
       'LPG 하이브리드', '수소', 'LNG(액화천연가스)'], dtype=object)

In [17]:
src['연료'].value_counts(dropna=False)

경유             269767
휘발유            213823
NaN             42120
CNG(압축천연가스)      7192
LPG(액화석유가스)      4804
휘발유 하이브리드        3780
전기               1720
기타연료             1583
경유 하이브리드          319
CNG 하이브리드         105
수소                 78
LNG(액화천연가스)        23
LPG 하이브리드          19
등유                 18
알코올                13
태양열                10
Name: 연료, dtype: int64

In [27]:
src.loc[src['연료'].isnull() == True, ['제원관리번호', '연료', '차명', '자동차형식', '엔진형식']].isnull().sum()

제원관리번호        0
연료        42120
차명           27
자동차형식      2841
엔진형식      19159
dtype: int64

In [22]:
src.loc[src['연료'].isnull() == True, ['제원관리번호', '연료', '차명', '자동차형식', '엔진형식']].head()

,제원관리번호,연료,차명,자동차형식,엔진형식
0,B0710004800013305,NaN,두성26.2㎥벌크 트레일러,DS-BS2L-262A,-
1,B0710004900013305,NaN,두성38.1㎥벌크 트레일러,DS-BS3L-381A,-
2,B0710005000013306,NaN,두성41㎥벌크 트레일러,DS-BS4L-410A,-
3,B0710005300013305,NaN,두성25.8톤코일트레일러,DS-LS0A-258A,-
4,B0710005400013305,NaN,두성25.8톤리프팅코일트레일러,DS-LS0A-258B,-


In [6]:
temp = tdf.loc[tdf['연료'].isnull() == True, '차대번호']

In [8]:
for one in temp:
    tp = tin.loc[tin['차대번호'] == one, ['차대번호', '사용연료']]
    if tp.shape[0] != 0:
        print(tp)

                 차대번호 사용연료
20559969  KG69C151977  NaN
                차대번호 사용연료
8368801  PN53C100440  NaN
                차대번호 사용연료
5284462  02734200016  NaN
                 차대번호 사용연료
20725224  02704204188  NaN
                차대번호 사용연료
5495337  PN71C020664   경유
                 차대번호 사용연료
15559169  02714202543  NaN
                 차대번호 사용연료
13035879  02734201013  NaN
                 차대번호 사용연료
14972685  02734201015  NaN
                 차대번호 사용연료
10723519  02734201006  NaN
                차대번호 사용연료
2523520  02734201016  NaN
                 차대번호 사용연료
15187492  02734204317  NaN
                 차대번호 사용연료
15257455  02734203365  NaN
                 차대번호 사용연료
18011551  02714201146  NaN
                       차대번호 사용연료
16106451  KN9ENBZTZVDZ00001  NaN
                      차대번호 사용연료
8238273  KN9ENAZTZ2HZ00001  NaN
                차대번호 사용연료
21971662  PN71C02663  NaN
                 차대번호 사용연료
17724672  SL55C040306   경유
                 차대번호 사용연료
20794743  SL67S060500  NaN
                

KeyboardInterrupt: 

In [9]:
tdf.loc[tdf['차대번호'] == 'PN71C020664', '제원관리번호']

67    30311100000072
Name: 제원관리번호, dtype: object

In [30]:
tin.loc[tin['차대번호'] == '30111100000076', '사용연료']

KeyError: '제원관리번호'

In [10]:
tdf.loc[tdf['연료'].isnull() == True, ['제원관리번호', '연료', '차명', '자동차형식', '엔진형식']].head()

,제원관리번호,연료,차명,자동차형식,엔진형식
16,30411200000060,NaN,트레라,NaN,NaN
17,30441300000012,NaN,트레일러,NaN,NaN
29,30310100000044,NaN,도큐트레일러,TCE203,NaN
34,30481400000064,NaN,콘테이너샤시트레일러,FT20CCCA,NaN
37,30230100001371,NaN,트레일러,NaN,NaN


In [11]:
temp = tdf.loc[tdf['연료'].isnull() == True, '자동차형식']
for one in temp:
    tp = tdf.loc[tdf['자동차형식'] == one, ['자동차형식', '연료']]
    if tp.shape[0] != 0:
        print(tp)

      자동차형식    연료
29   TCE203   NaN
614  TCE203   NaN
615  TCE203   NaN
616  TCE203  기타연료
617  TCE203   NaN
618  TCE203  기타연료
       자동차형식   연료
34  FT20CCCA  NaN
     자동차형식   연료
19  HT252T   경유
20  HT252T   경유
21  HT252T   경유
22  HT252T   경유
25  HT252T   경유
27  HT252T   경유
28  HT252T   경유
61  HT252T  NaN
62  HT252T   경유
63  HT252T   경유
      자동차형식   연료
68  TLE402L  NaN
      자동차형식   연료
186    1942  NaN
11932  1942   경유
     자동차형식   연료
254     카고  NaN
9854    카고  NaN
    자동차형식   연료
276    66  NaN
     자동차형식   연료
445  CS20C  NaN
446  CS20C  NaN
447  CS20C  NaN


KeyboardInterrupt: 

In [13]:
temp = tdf.loc[tdf['연료'].isnull() == True, '엔진형식']
for one in temp:
    tp = tdf.loc[tdf['엔진형식'] == one, ['엔진형식', '연료']]
    if tp.shape[0] != 0:
        print(tp)

       엔진형식   연료
133   DA640   경유
174   DA640   경유
183   DA640  NaN
3932  DA640   경유
                엔진형식   연료
254   10-78-11-64609  NaN
9854  10-78-11-64609  NaN
          엔진형식   연료
276  077801946  NaN


KeyboardInterrupt: 

## 엔진형식

In [139]:
src['엔진형식'].isnull().sum()

51217

In [140]:
tdf.loc[tdf['엔진형식'].isnull() == True, ['엔진형식', '차명', '자동차형식']]

,엔진형식,차명,자동차형식
1,NaN,퍼브리카,UP26LV
8,NaN,코티나 디럭스,NaN
16,NaN,트레라,NaN
17,NaN,트레일러,NaN
18,NaN,미스비시샤시(세미트레일러),MS2301
...,...,...,...
29532644,NaN,씨티인터내셔널캠핑트레일러,FOCUS470-1800
29532813,NaN,씨티인터내셔널캠핑트레일러,FOCUS470-1800
29533150,NaN,MAZDA 2,NaN
29533362,NaN,BMW 3 SERIES 2020,NaN


In [142]:
tdf.loc[tdf['엔진형식'].isnull() == True, ['엔진형식', '차명', '자동차형식']].tail(20)

,엔진형식,차명,자동차형식
29530529,NaN,모두베스트메이크폴딩캠핑트레일러,BM.PALACE
29530530,NaN,차캐드캠핑트레일러,TR300HW
29530531,NaN,차캐드캠핑트레일러,tr300
29530532,NaN,차캐드캠핑트레일러,tr300
29531066,NaN,SUBARU TREZIA 2012,NaN
29531184,NaN,LEXUS ES,NaN
29531206,NaN,투데이40피트콤비컨테이너운송트레일러,TDMCBCC4-DB21S
29531304,NaN,MAZDA 2,NaN
29532154,NaN,JEEP CHEROKEE 2022,NaN
29532169,NaN,FORD FUSION 2017,NaN


In [44]:
tdf.loc[tdf['엔진형식'].isnull() == True, ['엔진형식', '차명', '자동차형식']].iloc[800:810]

,엔진형식,차명,자동차형식
4072,NaN,새한2.5톤카고,TLD54
4084,NaN,새한2.5톤카고,TLD54
4096,NaN,현대26톤평판트레일러,HTC-5B
4098,NaN,현대26톤평판트레일러,HTC-5B
4099,NaN,콘테이너샤시콤바인,HDH-40-CCC-35SS
4136,NaN,현대35톤평판트레일러,HTC-5
4138,NaN,현대35톤평판트레일러,HTC-5
4140,NaN,현대35톤평판트레일러,HTC-5
4141,NaN,스트리크트레일러샤시,NaN
4199,NaN,스트리크트레일러샤시,NaN


In [143]:
tdf.loc[tdf['차명'] == 'LEXUS ES', ['엔진형식', '차명', '자동차형식']]

,엔진형식,차명,자동차형식
24424020,NaN,LEXUS ES,NaN
27631663,2GR,LEXUS ES,ES350
29531184,NaN,LEXUS ES,NaN


# 3\. 배출가스 인증정보

In [6]:
name = 'CEG_EMIS_CERTI_MIG'
file_name = f'ODS_{name}.csv'
emis_certi_mig = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
emis_certi_mig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79725 entries, 0 to 79724
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   EMIS_CRTCNO            79725 non-null  object 
 1   EMIS_CRTC_SNO          79725 non-null  int64  
 2   EMIS_CRTC_1_SE         79725 non-null  object 
 3   EMIS_CRTC_2_SE         79436 non-null  object 
 4   DMSTC_INCM_MKR_SE      79038 non-null  object 
 5   EMIS_VHCTY             79519 non-null  object 
 6   FUEL                   79493 non-null  object 
 7   SAME_VHCTY_TY          9155 non-null   object 
 8   EMIS_CRTC_DE           79670 non-null  float64
 9   MKR_NM                 78956 non-null  object 
 10  MNFCT_NM               79554 non-null  object 
 11  PLOR_NM                29928 non-null  object 
 12  EMIS_CRTC_STUS         79681 non-null  float64
 13  EMIS_CRTC_UNAVBL_DE    361 non-null    float64
 14  EMIS_CRTC_UNAVBL_RESN  45 non-null     object 
 15  OR

In [7]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
emis_certi_mig.columns = [col_dict[x] for x in emis_certi_mig.columns]
emis_certi_mig.columns

Index(['배출가스인증번호', '배출가스 인증 일련번호', '배출가스 인증 1 구분', '배출가스 인증 2 구분',
       '국내 수입 제작사 구분', '배출가스 차종', '연료', '동일 차종 유형', '배출가스 인증 일자', '제작사 명',
       '제조사 명', '원산지 명', '배출가스 인증 상태', '배출가스 인증 무효 일자', '배출가스 인증 무효 사유',
       '기본차 제원관리번호', '대표차 배출가스 인증번호', '대표차 차대번호', '배출가스 허용 기준 코드', '배출가스 등급',
       '미정', '저장 일자', '미정'],
      dtype='object')

In [8]:
emis_certi_mig1 = emis_certi_mig[['배출가스인증번호', '배출가스 인증 일련번호', '배출가스 인증 1 구분', '배출가스 인증 2 구분',
       '국내 수입 제작사 구분', '배출가스 차종', '연료', '동일 차종 유형', '배출가스 인증 일자', '제작사 명',
       '제조사 명', '원산지 명', '배출가스 인증 상태', '배출가스 인증 무효 일자', '배출가스 인증 무효 사유',
       '기본차 제원관리번호', '대표차 배출가스 인증번호', '대표차 차대번호', '배출가스 허용 기준 코드', '배출가스 등급',
       '저장 일자']]

In [15]:
col = pd.DataFrame(col_dict, index=['내용']).T.reset_index()
col.columns = ['컬럼명', '내용']
col

,컬럼명,내용
0,STRE_DE,저장 일자
1,MKR_NM,제작사 명
2,FUEL,연료
3,EMIS_CRTC_DE,배출가스 인증 일자
4,EMIS_CRTCNO,배출가스인증번호
5,EMIS_PERM_STDR_CD,배출가스 허용 기준 코드
6,EMIS_VHCTY,배출가스 차종
7,EMIS_CRTC_UNAVBL_RESN,배출가스 인증 무효 사유
8,EMIS_CRTC_2_SE,배출가스 인증 2 구분
9,EMIS_CRTC_STUS,배출가스 인증 상태


In [20]:
nul = emis_certi_mig1.isnull().sum().reset_index()
nul.columns = ['내용', '결측 값']
nul

,내용,결측 값
0,배출가스인증번호,0
1,배출가스 인증 일련번호,0
2,배출가스 인증 1 구분,0
3,배출가스 인증 2 구분,289
4,국내 수입 제작사 구분,687
5,배출가스 차종,206
6,연료,232
7,동일 차종 유형,70570
8,배출가스 인증 일자,55
9,제작사 명,769


In [22]:
col2 = col.merge(nul, on='내용', how='left')
col2

,컬럼명,내용,결측 값
0,STRE_DE,저장 일자,10.00
1,MKR_NM,제작사 명,769.00
2,FUEL,연료,232.00
3,EMIS_CRTC_DE,배출가스 인증 일자,55.00
4,EMIS_CRTCNO,배출가스인증번호,0.00
5,EMIS_PERM_STDR_CD,배출가스 허용 기준 코드,61979.00
6,EMIS_VHCTY,배출가스 차종,206.00
7,EMIS_CRTC_UNAVBL_RESN,배출가스 인증 무효 사유,79680.00
8,EMIS_CRTC_2_SE,배출가스 인증 2 구분,289.00
9,EMIS_CRTC_STUS,배출가스 인증 상태,44.00


# 4\. 배출가스 인증시험

In [23]:
name = 'CEG_EMIS_CERTI_TEST_MIG'
file_name = f'ODS_{name}.csv'
emis_certi_test_mig = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
emis_certi_test_mig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86406 entries, 0 to 86405
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   EMIS_CRTCNO             86406 non-null  object 
 1   EMIS_CRTC_SNO           86406 non-null  int64  
 2   EMIS_TEST_MOD_VAL       85442 non-null  object 
 3   CO_PERM_STDR_VAL        16761 non-null  float64
 4   CO_TEST_RSLT            17868 non-null  float64
 5   NMOG_PERM_STDR_VAL      43 non-null     float64
 6   NMOG_TEST_RSLT          3116 non-null   float64
 7   NMHC_PERM_STDR_VAL      20 non-null     float64
 8   NMHC_TEST_RSLT          268 non-null    float64
 9   HC_PERM_STDR_VAL        12176 non-null  float64
 10  HC_TEST_RSLT            12236 non-null  float64
 11  NOX_PERM_STDR_VAL       13893 non-null  float64
 12  NOX_TEST_RSLT           16856 non-null  float64
 13  NMOG_NOX_PERM_STDR_VAL  4229 non-null   float64
 14  NMOG_NOX_TEST_RSLT      4783 non-null 

In [24]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
emis_certi_test_mig.columns = [col_dict[x] for x in emis_certi_test_mig.columns]
emis_certi_test_mig.columns

Index(['배출가스 인증번호', '배출가스 인증 일련번호', '배출가스 시험 모드 값', '일산화탄소 허용 기준 값',
       '일산화탄소 시험 결과', '탄화수소1 허용 기준 값', '탄화수소1 시험 결과', '탄화수소2 허용 기준 값',
       '탄화수소2 시험 결과', '탄화수소3 허용 기준 값', '탄화수소3 시험 결과', '질소산화물 허용 기준 값',
       '질소산화물 시험 결과', '탄화수소1 질소산화물 허용 기준 값', '탄화수소1 질소산화물 시험 결과',
       '탄화수소3 질소산화물 허용 기준 값', '탄화수소3 질소산화물 시험 결과', '입자상물질 허용 기준 값',
       '입자상물질 시험 결과', '입자개수 허용 기준 값', '입자개수 시험 결과', '미정', '미정'],
      dtype='object')

In [25]:
emis_certi_test_mig1 = emis_certi_test_mig[['배출가스 인증번호', '배출가스 인증 일련번호', '배출가스 시험 모드 값', '일산화탄소 허용 기준 값',
       '일산화탄소 시험 결과', '탄화수소1 허용 기준 값', '탄화수소1 시험 결과', '탄화수소2 허용 기준 값',
       '탄화수소2 시험 결과', '탄화수소3 허용 기준 값', '탄화수소3 시험 결과', '질소산화물 허용 기준 값',
       '질소산화물 시험 결과', '탄화수소1 질소산화물 허용 기준 값', '탄화수소1 질소산화물 시험 결과',
       '탄화수소3 질소산화물 허용 기준 값', '탄화수소3 질소산화물 시험 결과', '입자상물질 허용 기준 값',
       '입자상물질 시험 결과', '입자개수 허용 기준 값', '입자개수 시험 결과']]

In [26]:
emis_certi_test_mig1.isnull().sum()

배출가스 인증번호                  0
배출가스 인증 일련번호               0
배출가스 시험 모드 값             964
일산화탄소 허용 기준 값          69645
일산화탄소 시험 결과            68538
탄화수소1 허용 기준 값          86363
탄화수소1 시험 결과            83290
탄화수소2 허용 기준 값          86386
탄화수소2 시험 결과            86138
탄화수소3 허용 기준 값          74230
탄화수소3 시험 결과            74170
질소산화물 허용 기준 값          72513
질소산화물 시험 결과            69550
탄화수소1 질소산화물 허용 기준 값    82177
탄화수소1 질소산화물 시험 결과      81623
탄화수소3 질소산화물 허용 기준 값    77893
탄화수소3 질소산화물 시험 결과      77858
입자상물질 허용 기준 값          83636
입자상물질 시험 결과            79946
입자개수 허용 기준 값           84428
입자개수 시험 결과             86406
dtype: int64

In [27]:
col = pd.DataFrame(col_dict, index=['내용']).T.reset_index()
col.columns = ['컬럼명', '내용']
nul = emis_certi_test_mig1.isnull().sum().reset_index()
nul.columns = ['내용', '결측 값']
col2 = col.merge(nul, on='내용', how='left')
col2

,컬럼명,내용,결측 값
0,PN_PERM_STDR_VAL,입자개수 허용 기준 값,84428.00
1,NOX_PERM_STDR_VAL,질소산화물 허용 기준 값,72513.00
2,PM_PERM_STDR_VAL,입자상물질 허용 기준 값,83636.00
3,NOX_TEST_RSLT,질소산화물 시험 결과,69550.00
4,PN_TEST_RSLT,입자개수 시험 결과,86406.00
5,CO_PERM_STDR_VAL,일산화탄소 허용 기준 값,69645.00
6,CO_TEST_RSLT,일산화탄소 시험 결과,68538.00
7,EMIS_CRTCNO,배출가스 인증번호,0.00
8,EMIS_CRTC_SNO,배출가스 인증 일련번호,0.00
9,EMIS_TEST_MOD_VAL,배출가스 시험 모드 값,964.00


# 5\. 배출가스인증 자동차정보

In [102]:
name = 'CEG_EMIS_CERTI_VEH_MIG'
file_name = f'ODS_{name}.csv'
emis_certi_veh_mig = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
emis_certi_test_mig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86406 entries, 0 to 86405
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   배출가스 인증번호            86406 non-null  object 
 1   배출가스 인증 일련번호         86406 non-null  int64  
 2   배출가스 시험 모드 값         85442 non-null  object 
 3   일산화탄소 허용 기준 값        16761 non-null  float64
 4   일산화탄소 시험 결과          17868 non-null  float64
 5   탄화수소1 허용 기준 값        43 non-null     float64
 6   탄화수소1 시험 결과          3116 non-null   float64
 7   탄화수소2 허용 기준 값        20 non-null     float64
 8   탄화수소2 시험 결과          268 non-null    float64
 9   탄화수소3 허용 기준 값        12176 non-null  float64
 10  탄화수소3 시험 결과          12236 non-null  float64
 11  질소산화물 허용 기준 값        13893 non-null  float64
 12  질소산화물 시험 결과          16856 non-null  float64
 13  탄화수소1 질소산화물 허용 기준 값  4229 non-null   float64
 14  탄화수소1 질소산화물 시험 결과    4783 non-null   float64
 15  탄화수소3 질소산화물 허용 기준 값  8513 non-null  

In [103]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
emis_certi_veh_mig.columns = [col_dict[x] for x in emis_certi_veh_mig.columns]
emis_certi_veh_mig.columns

Index(['배출가스 인증번호', '배출가스 인증 일련번호', '차량 일련번호', '차명', '자동차 형식', '차대번호',
       '배출가스 차종', '차량 연식', '구동 형식', '용도', '변속기 종류', '공차중량', '총 중량', '엔진형식',
       '배기량', '연료', '엔진출력', '차량 길이 정보', '차량 너비 정보', '차량 높이 정보', '승차 인원 정보',
       '미정', '미정', '미정', '저장 일자'],
      dtype='object')

In [104]:
emis_certi_veh_mig1 = emis_certi_veh_mig[['배출가스 인증번호', '배출가스 인증 일련번호', '차량 일련번호', '차명', '자동차 형식', '차대번호',
       '배출가스 차종', '차량 연식', '구동 형식', '용도', '변속기 종류', '공차중량', '총 중량', '엔진형식',
       '배기량', '연료', '엔진출력', '차량 길이 정보', '차량 너비 정보', '차량 높이 정보', '승차 인원 정보',
       '저장 일자']]

In [105]:
emis_certi_veh_mig1.isnull().sum()

배출가스 인증번호           0
배출가스 인증 일련번호        0
차량 일련번호             0
차명                 28
자동차 형식             88
차대번호            36771
배출가스 차종         25228
차량 연식            4937
구동 형식           18127
용도              12319
변속기 종류            416
공차중량              103
총 중량              242
엔진형식              990
배기량              1020
연료                523
엔진출력             9594
차량 길이 정보         6610
차량 너비 정보         6607
차량 높이 정보         6609
승차 인원 정보        26770
저장 일자               4
dtype: int64

In [106]:
col = pd.DataFrame(col_dict, index=['내용']).T.reset_index()
col.columns = ['컬럼명', '내용']
nul = emis_certi_veh_mig1.isnull().sum().reset_index()
nul.columns = ['내용', '결측 값']
col2 = col.merge(nul, on='내용', how='left')
display(col2)

,컬럼명,내용,결측 값
0,EMIS_VHCTY,배출가스 차종,25228.00
1,EMPV_WGHT,공차중량,103.00
2,PURPS,용도,12319.00
3,EMIS_CRTCNO,배출가스 인증번호,0.00
4,EMIS_CRTC_SNO,배출가스 인증 일련번호,0.00
5,DSPLVL,배기량,1020.00
6,DRV_FRM,구동 형식,18127.00
7,FUEL,연료,523.00
8,EGINTY,엔진형식,990.00
9,EGINPWR,엔진출력,9594.00


# 6\. 제작차 배출 허용 기준 

In [33]:
name = 'CEG_EMIS_STD_MIG'
file_name = f'ODS_{name}.csv'
emis_std_mig = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
emis_std_mig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1212 entries, 0 to 1211
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   EMIS_PERM_STDR_CD       1212 non-null   int64  
 1   EMIS_PERM_STDR_SML_SE   1212 non-null   object 
 2   FUEL_GRP                1212 non-null   int64  
 3   EMIS_VHCTY              1212 non-null   object 
 4   TEST_BGIN_WGHT          1212 non-null   object 
 5   TEST_END_WGHT           1212 non-null   object 
 6   APLI_DE                 1212 non-null   object 
 7   EMIS_TEST_MOD_VAL       1212 non-null   object 
 8   CO_PERM_STDR_VAL        1182 non-null   float64
 9   NMOG_PERM_STDR_VAL      212 non-null    float64
 10  NMHC_PERM_STDR_VAL      374 non-null    float64
 11  HC_PERM_STDR_VAL        125 non-null    float64
 12  NOX_PERM_STDR_VAL       884 non-null    float64
 13  NMOG_NOX_PERM_STDR_VAL  328 non-null    float64
 14  HC_NOX_PERM_STDR_VAL    151 non-null    

In [34]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
emis_std_mig.columns = [col_dict[x] for x in emis_std_mig.columns]
emis_std_mig.columns

Index(['배출가스 허용 기준 코드', '배출가스 허용 기준 소 구분', '연료 그룹', '배출가스 차종', '시험 시작 중량',
       '시험 끝 중량', '적용 일자', '배출가스 시험 모드 값', '일산화탄소 허용 기준 값', '탄화수소1 허용 기준 값',
       '탄화수소2 허용 기준 값', '탄화수소3 허용기준 값', '질소산화물 허용 기준 값', '탄화수소1 질소산화물 허용 기준 값',
       '탄화수소3 질소산화물 허용 기준 값', '입자상물질 허용 기준 값', '입자개수 허용 기준 값', '배출가스 허용 기준 구분',
       '배출가스 등급 사용 YN', '배출가스 등급', '배출가스 허용 기준 수준'],
      dtype='object')

In [35]:
emis_std_mig1 = emis_std_mig[['배출가스 허용 기준 코드', '배출가스 허용 기준 소 구분', '연료 그룹', '배출가스 차종', '시험 시작 중량',
       '시험 끝 중량', '적용 일자', '배출가스 시험 모드 값', '일산화탄소 허용 기준 값', '탄화수소1 허용 기준 값',
       '탄화수소2 허용 기준 값', '탄화수소3 허용기준 값', '질소산화물 허용 기준 값', '탄화수소1 질소산화물 허용 기준 값',
       '탄화수소3 질소산화물 허용 기준 값', '입자상물질 허용 기준 값', '입자개수 허용 기준 값', '배출가스 허용 기준 구분',
       '배출가스 등급 사용 YN', '배출가스 등급', '배출가스 허용 기준 수준']]

In [36]:
emis_std_mig1.isnull().sum()

배출가스 허용 기준 코드             0
배출가스 허용 기준 소 구분           0
연료 그룹                     0
배출가스 차종                   0
시험 시작 중량                  0
시험 끝 중량                   0
적용 일자                     0
배출가스 시험 모드 값              0
일산화탄소 허용 기준 값            30
탄화수소1 허용 기준 값          1000
탄화수소2 허용 기준 값           838
탄화수소3 허용기준 값           1087
질소산화물 허용 기준 값           328
탄화수소1 질소산화물 허용 기준 값     884
탄화수소3 질소산화물 허용 기준 값    1061
입자상물질 허용 기준 값           561
입자개수 허용 기준 값           1148
배출가스 허용 기준 구분             0
배출가스 등급 사용 YN           146
배출가스 등급                 146
배출가스 허용 기준 수준           156
dtype: int64

In [37]:
col = pd.DataFrame(col_dict, index=['내용']).T.reset_index()
col.columns = ['컬럼명', '내용']
nul = emis_std_mig1.isnull().sum().reset_index()
nul.columns = ['내용', '결측 값']
col2 = col.merge(nul, on='내용', how='left')
col2

,컬럼명,내용,결측 값
0,APLI_DE,적용 일자,0
1,TEST_END_WGHT,시험 끝 중량,0
2,TEST_BGIN_WGHT,시험 시작 중량,0
3,PN_PERM_STDR_VAL,입자개수 허용 기준 값,1148
4,PM_PERM_STDR_VAL,입자상물질 허용 기준 값,561
5,CO_PERM_STDR_VAL,일산화탄소 허용 기준 값,30
6,NMOG_PERM_STDR_VAL,탄화수소1 허용 기준 값,1000
7,NOX_PERM_STDR_VAL,질소산화물 허용 기준 값,328
8,EMIS_GRD,배출가스 등급,146
9,EMIS_GRD_USE_YN,배출가스 등급 사용 YN,146


# 7\. 대표차명

In [38]:
name = 'CEG_STD_CAR_MIG'
file_name = f'ODS_{name}.csv'
std_car_mig = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
std_car_mig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1541 entries, 0 to 1540
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   REPCAR_NM  1541 non-null   object
 1   MKR_NM     1541 non-null   object
 2   PLOR_NM    1541 non-null   object
 3   RMK        90 non-null     object
 4   CRTN_DT    90 non-null     object
 5   UPDT_DT    6 non-null      object
 6   USER_ID    1541 non-null   object
dtypes: object(7)
memory usage: 84.4+ KB


In [39]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
std_car_mig.columns = [col_dict[x] for x in std_car_mig.columns]
std_car_mig.columns

Index(['대표차 명', '제작사 명', '원산지 명', '비고', '생성 일시', '수정 일시', '사용자 아이디'], dtype='object')

In [40]:
std_car_mig1 = std_car_mig[['대표차 명', '제작사 명', '원산지 명', '비고', '생성 일시', '수정 일시', '사용자 아이디']]

In [41]:
std_car_mig1.isnull().sum()

대표차 명         0
제작사 명         0
원산지 명         0
비고         1451
생성 일시      1451
수정 일시      1535
사용자 아이디       0
dtype: int64

In [42]:
col = pd.DataFrame(col_dict, index=['내용']).T.reset_index()
col.columns = ['컬럼명', '내용']
nul = std_car_mig1.isnull().sum().reset_index()
nul.columns = ['내용', '결측 값']
col2 = col.merge(nul, on='내용', how='left')
col2

,컬럼명,내용,결측 값
0,USER_ID,사용자 아이디,0
1,UPDT_DT,수정 일시,1535
2,RMK,비고,1451
3,REPCAR_NM,대표차 명,0
4,PLOR_NM,원산지 명,0
5,MKR_NM,제작사 명,0
6,CRTN_DT,생성 일시,1451


# 8\. 저감장치 클리닝 상세 정보

In [43]:
name = 'DLM_TB_ERP_CLEANING_DTL'
file_name = f'ODS_{name}.csv'
cleaning_dtl = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
cleaning_dtl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 896011 entries, 0 to 896010
Data columns (total 59 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   REDUC_REG_NO            896002 non-null  float64
 1   CLNING_MNGT_NO          896011 non-null  int64  
 2   MKR_CD                  45588 non-null   object 
 3   CLNING_DE               896009 non-null  float64
 4   CLNING_KND              896007 non-null  object 
 5   CLNING_CST              896010 non-null  float64
 6   CLNING_ENTRPS_NM        896007 non-null  object 
 7   DRVNG_DSTNC             896007 non-null  object 
 8   CLNING_PRGRE_STUS       896011 non-null  object 
 9   SBSIDY_TRGT_YN          896010 non-null  object 
 10  SBSIDY_TRGT_EXCLU_RESN  33 non-null      object 
 11  LOCGOV_CNFRM_DE         718007 non-null  float64
 12  LOCGOV_CNFRMR_NM        718007 non-null  object 
 13  MKR_PAPE_ATT_YN         896011 non-null  object 
 14  SBSIDY_PYMT_DCSN_DE 

In [44]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
cleaning_dtl.columns = [col_dict[x] for x in cleaning_dtl.columns]
cleaning_dtl.columns

Index(['저감등록번호', '미정', '미정', '클리닝일자', '클리닝종류', '미정', '클리닝업체명', '주행거리',
       '클리닝진행상태', '보조금대상YN', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정',
       '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '전매연농도', '후매연농도',
       '전배기압력', '후배기압력', '미정', '미정', '미정', '미정', '미정', '미정', '클리닝보증기간구분', '미정',
       '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정',
       '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정'],
      dtype='object')

In [45]:
cleaning_dtl1 = cleaning_dtl[['저감등록번호', '클리닝일자', '클리닝종류', '클리닝업체명', '주행거리', 
                              '클리닝진행상태', '보조금대상YN', '전매연농도', '후매연농도', '전배기압력', '후배기압력', '클리닝보증기간구분']]

In [46]:
cleaning_dtl1.isnull().sum()

저감등록번호        9
클리닝일자         2
클리닝종류         4
클리닝업체명        4
주행거리          4
클리닝진행상태       0
보조금대상YN       1
전매연농도         1
후매연농도         1
전배기압력         1
후배기압력         1
클리닝보증기간구분    10
dtype: int64

In [47]:
col = pd.DataFrame(col_dict, index=['내용']).T.reset_index()
col.columns = ['컬럼명', '내용']
nul = cleaning_dtl1.isnull().sum().reset_index()
nul.columns = ['내용', '결측 값']
col2 = col.merge(nul, on='내용', how='left')
col2

,컬럼명,내용,결측 값
0,CLNING_GRNTY_WHL_SE,클리닝보증기간구분,10.00
1,CLNING_KND,클리닝종류,4.00
2,CLNING_ENTRPS_NM,클리닝업체명,4.00
3,CLNING_DE,클리닝일자,2.00
4,SBSIDY_TRGT_YN,보조금대상YN,1.00
5,BF_SMO_DNSTY,전매연농도,1.00
6,BF_VENTL_PSSR,전배기압력,1.00
7,REDUC_REG_NO,저감등록번호,9.00
8,PST_VENTL_PSSR,후배기압력,1.00
9,CLNING_PRGRE_STUS,클리닝진행상태,0.00


# 9\. 사후관리 사후점검

In [48]:
name = 'DLM_TB_ERP_CLEAN_SCREEN'
file_name = f'ODS_{name}.csv'
clean_screen = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
clean_screen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64951 entries, 0 to 64950
Data columns (total 40 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CHK_RCPT_NO          64951 non-null  int64  
 1   REDUC_REG_NO         64951 non-null  int64  
 2   RCPT_DE              45629 non-null  float64
 3   CHK_DE               64951 non-null  int64  
 4   CHK_ODR              50015 non-null  object 
 5   RCPT_SIGNGU          64885 non-null  object 
 6   EXTRNS_REMDEL_STUS   64951 non-null  object 
 7   SLF_DGNSS_DVICE      64569 non-null  object 
 8   FUEL_SPLORE_PICK     50276 non-null  object 
 9   PLLTN_DNSTY_MEASUVL  62121 non-null  object 
 10  SMO_CO_VAL           210 non-null    object 
 11  SMO_HC_VAL           169 non-null    float64
 12  CHK_RSLT             64890 non-null  object 
 13  DIRTN_MTTR           11070 non-null  object 
 14  RCPT_RESN_01_YN      1109 non-null   object 
 15  RCPT_RESN_02_YN      43 non-null    

In [49]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
clean_screen.columns = [col_dict[x] for x in clean_screen.columns]
clean_screen.columns

Index(['미정', '저감등록번호', '미정', '점검일자', '점검차수', '미정', '부착개조상태', '미정', '미정',
       '오염농도측정값', '미정', '미정', '점검결과', '지시사항', '미정', '미정', '미정', '미정', '미정',
       '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정',
       '미정', '미정', '미정', '미정', '미정', '미정', '미정', '조치내역', '미정'],
      dtype='object')

In [50]:
clean_screen1 = clean_screen[['저감등록번호', '점검일자', '점검차수', '부착개조상태', '오염농도측정값', '점검결과', '지시사항', '조치내역']]

In [51]:
clean_screen1.isnull().sum()

저감등록번호         0
점검일자           0
점검차수       14936
부착개조상태         0
오염농도측정값     2830
점검결과          61
지시사항       53881
조치내역       63254
dtype: int64

In [52]:
col = pd.DataFrame(col_dict, index=['내용']).T.reset_index()
col.columns = ['컬럼명', '내용']
nul = clean_screen1.isnull().sum().reset_index()
nul.columns = ['내용', '결측 값']
col2 = col.merge(nul, on='내용', how='left')
col2

,컬럼명,내용,결측 값
0,CHK_DE,점검일자,0.00
1,CHK_ODR,점검차수,14936.00
2,TRTMNT_DTLS,조치내역,63254.00
3,DIRTN_MTTR,지시사항,53881.00
4,PLLTN_DNSTY_MEASUVL,오염농도측정값,2830.00
5,REDUC_REG_NO,저감등록번호,0.00
6,CHK_RSLT,점검결과,61.00
7,EXTRNS_REMDEL_STUS,부착개조상태,0.00
8,CHK_RCPT_NO,미정,NaN
9,RCPT_DE,미정,NaN


# 10\. 노후차 조기폐차 관리정보(수도권)

In [39]:
name = 'DLM_TB_ERP_EARLY_ERASE_AEA'
file_name = f'ODS_{name}.csv'
early_aea = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
early_aea.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770311 entries, 0 to 770310
Columns: 177 entries, ELPDSRC_MNGT_NO to PFMINPC_ELPDSRC_GNRLZ_RESN
dtypes: float64(56), int64(3), object(118)
memory usage: 1.0+ GB


In [40]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
early_aea.columns = [col_dict[x] for x in early_aea.columns]
early_aea.columns

Index(['미정', '조기폐차상태코드', '저감장치구분', '미정', '미정', '미정', '미정', '미정', '미정', '미정',
       ...
       '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정'],
      dtype='object', length=177)

In [42]:
early_aea[['차대번호', '저감장치구분', '차량등록번호', '조기폐차최종승인YN', '조기폐차상태코드', '말소일자', '접수시도', '저감장치미개발(부착불가)여부', '폐차등록주행거리']].head()

,차대번호,저감장치구분,차량등록번호,조기폐차최종승인YN,조기폐차상태코드,말소일자,접수시도,저감장치미개발(부착불가)여부,폐차등록주행거리
0,KMYKF17APRU074823,조기폐차,73부5582,Y,Y,20100317.00,인천시,NaN,234306.00
1,KMFFE17APSU290436,조기폐차,93소6048,NaN,B,NaN,인천시,NaN,401697.00
2,KMJWWH7BPXU115199,조기폐차,경기74도2843,Y,Y,20100310.00,경기도,NaN,70216.00
3,KPDHADMD1XP166822,조기폐차,39다9380,NaN,D,NaN,경기도,NaN,164840.00
4,KMFXKD7BPYU393462,조기폐차,서울85도4906,Y,Y,20100310.00,서울시,NaN,119789.00


In [6]:
early_aea1 = early_aea[['차대번호', '저감장치구분']]
early_aea1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770311 entries, 0 to 770310
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   차대번호    770311 non-null  object
 1   저감장치구분  770311 non-null  object
dtypes: object(2)
memory usage: 11.8+ MB


## 차대번호

In [9]:
len_not_17_idx = []
len_not_17 = []
for i, one in enumerate(early_aea1['차대번호']):
    if len(str(one)) != 17:
        len_not_17_idx.append(i)
        len_not_17.append(len(str(one)))
len(len_not_17_idx)

1633

In [10]:
set(len_not_17)

{3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 20, 27}

In [11]:
len_not_17_temp = early_aea1.loc[len_not_17_idx]
len_not_17_temp.shape

(1633, 2)

In [12]:
len_not_17_temp['len_vin'] = len_not_17

In [19]:
for one in set(len_not_17):
    display(len_not_17_temp[len_not_17_temp['len_vin'] == one].head())

,차대번호,저감장치구분,len_vin
67445,ㅎㄻㅎ,조기폐차,3
734831,107,조기폐차,3
738014,107,조기폐차,3


,차대번호,저감장치구분,len_vin
396107,02153,조기폐차,5
434612,00045,조기폐차,5
435322,03631,조기폐차,5
579437,03529,조기폐차,5
640721,03480,조기폐차,5


,차대번호,저감장치구분,len_vin
394137,000661,조기폐차,6
399563,000663,조기폐차,6
400239,006721,조기폐차,6
434428,006948,조기폐차,6
435160,103843,조기폐차,6


,차대번호,저감장치구분,len_vin
389955,3BY0509,조기폐차,7
466284,3CY0194,조기폐차,7
479660,3EY0293,조기폐차,7
480380,3EY0185,조기폐차,7
543605,3EY0293,조기폐차,7


,차대번호,저감장치구분,len_vin
467742,34492468,조기폐차,8
516284,75101571,조기폐차,8
524627,DCP32022,조기폐차,8
601256,71-02860,조기폐차,8
613591,75101571,조기폐차,8


...

,차대번호,저감장치구분,len_vin
378221,FV415JML-100879,조기폐차,15
384766,FV415JML-005368,조기폐차,15
392514,FV415JML-102042,조기폐차,15
392585,FV415JML-100622,조기폐차,15
392982,FV415JML-101172,조기폐차,15


,차대번호,저감장치구분,len_vin
39322,KPAEA4MDXP173236,조기폐차,16
64704,KNAMA75131S15976,조기폐차,16
68120,KMHMH81VP2U12928,조기폐차,16
105067,41-94-342-80868수,조기폐차,16
116078,KMFWVH7HP6U70128,조기폐차,16


,차대번호,저감장치구분,len_vin
86205,KN90J4MDDD7HZ00003,조기폐차,18
392494,KNRB4AMAZVC-001262,조기폐차,18
399496,KL3RG58D32K-000320,조기폐차,18
399525,KNRB4AMA-ZVC001150,조기폐차,18
410297,KNRB4AMAZSC-000225,조기폐차,18


,차대번호,저감장치구분,len_vin
733361,SM510HWS=06M0-000023,조기폐차,20


,차대번호,저감장치구분,len_vin
711014,서울06자6935(KNRB46MAZVC003654,조기폐차,27
726383,서울06자6935(KNRB46MAZVC003654,조기폐차,27


In [21]:
len_17_wrong_text_idx = []
p = re.compile('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]')
for i, one in enumerate(early_aea1['차대번호']):
    if len(str(one)) == 17 and p.search(str(one)):
        len_17_wrong_text_idx.append(i)
len(len_17_wrong_text_idx)

0

## 저감장치구분

In [22]:
early_aea1['저감장치구분'].unique()

array(['조기폐차'], dtype=object)

# 11\. 노후차 조기폐차 관리정보(수도권 외)

In [43]:
name = 'DLM_TB_ERP_EARLY_ERASE_LGV'
file_name = f'ODS_{name}.csv'
early_lgv = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
early_lgv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291353 entries, 0 to 291352
Columns: 170 entries, ELPDSRC_MNGT_NO to PFMINPC_ELPDSRC_GNRLZ_RESN
dtypes: float64(71), int64(4), object(95)
memory usage: 377.9+ MB


In [44]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
early_lgv.columns = [col_dict[x] for x in early_lgv.columns]
early_lgv.columns

Index(['미정', '조기폐차상태코드', '저감장치구분', '미정', '미정', '미정', '미정', '미정', '미정', '미정',
       ...
       '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정'],
      dtype='object', length=170)

In [45]:
early_lgv[['차대번호', '저감장치구분', '차량등록번호', '조기폐차최종승인YN', '조기폐차상태코드', '말소일자', '접수시도', '저감장치 부착불가 여부', '폐차등록주행거리']].head()

,차대번호,저감장치구분,차량등록번호,조기폐차최종승인YN,조기폐차상태코드,말소일자,접수시도,저감장치 부착불가 여부,폐차등록주행거리
0,KNAJE55136K177805,조기폐차,60다6989,Y,Y,20210316.00,강원도,N,200356.00
1,KMHSD81VP4U723925,조기폐차,50모3406,Y,Y,20210317.00,강원도,Y,235898.00
2,KMHSD81VP4U657262,조기폐차,35마2904,Y,Y,20210409.00,강원도,N,313465.00
3,KN3HRP4N35K087584,조기폐차,95저3210,Y,Y,20210312.00,강원도,Y,159242.00
4,KN3HDP3363K918987,조기폐차,강원80구4284,Y,Y,20210304.00,강원도,Y,150566.00


In [25]:
early_lgv1 = early_lgv[['차대번호', '저감장치구분']]
early_lgv1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291353 entries, 0 to 291352
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   차대번호    291353 non-null  object
 1   저감장치구분  291353 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB


## 차대번호

In [26]:
len_not_17_idx = []
len_not_17 = []
for i, one in enumerate(early_lgv1['차대번호']):
    if len(str(one)) != 17:
        len_not_17_idx.append(i)
        len_not_17.append(len(str(one)))
len(len_not_17_idx)

1153

In [27]:
set(len_not_17)

{4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20}

In [28]:
len_not_17_temp = early_lgv1.loc[len_not_17_idx]
len_not_17_temp.shape

(1153, 2)

In [29]:
len_not_17_temp['len_vin'] = len_not_17

In [30]:
for one in set(len_not_17):
    display(len_not_17_temp[len_not_17_temp['len_vin'] == one].head())

,차대번호,저감장치구분,len_vin
212626,0192,조기폐차,4


,차대번호,저감장치구분,len_vin
191537,01132,조기폐차,5
197204,03120,조기폐차,5
246037,00342,조기폐차,5
265160,74102,조기폐차,5


,차대번호,저감장치구분,len_vin
184253,000189,조기폐차,6
184790,002568,조기폐차,6
186893,002218,조기폐차,6
191734,000584,조기폐차,6
192246,004381,조기폐차,6


,차대번호,저감장치구분,len_vin
78510,3EY0232,조기폐차,7
121515,3EY0155,조기폐차,7
136344,3AY0155,조기폐차,7
137030,3EY0092,조기폐차,7
159471,3CY0281,조기폐차,7


,차대번호,저감장치구분,len_vin
136345,DCP36070,조기폐차,8
159470,DCP32101,조기폐차,8
162641,DCP32176,조기폐차,8
163583,DCP36073,조기폐차,8
165518,DCP32281,조기폐차,8


...

,차대번호,저감장치구분,len_vin
25159,KPAKAD12P311802,조기폐차,15
54351,KMFWVHP6U696210,조기폐차,15
114317,FV415JML-003864,조기폐차,15
116695,FV415JML-007303,조기폐차,15
116760,FV415JML-004781,조기폐차,15


,차대번호,저감장치구분,len_vin
9146,XLEP840014453260,조기폐차,16
21162,KPAKA4D13P338347,조기폐차,16
38845,KNAFD52424372154,조기폐차,16
38940,KPBF2AD12P047564,조기폐차,16
67185,KMFXKSBP2U623023,조기폐차,16


,차대번호,저감장치구분,len_vin
120009,DTW015L1-75-108215,조기폐차,18
143860,KL3MA45D1TK-001094,조기폐차,18
144951,AU745L-6-71-102583,조기폐차,18
168513,KL3MD45DIVK-000450,조기폐차,18
168740,KL3MC45M1XK-000275,조기폐차,18


,차대번호,저감장치구분,len_vin
173706,kKL3MA45D1YK-003471,조기폐차,19
192435,KL3MC 45M1VK 000482,조기폐차,19
270315,KMCDB 18YPVC 105027,조기폐차,19
273184,KMCMCDH19SP2C020162,조기폐차,19


,차대번호,저감장치구분,len_vin
199239,KL3K4MVA1212K-000106,조기폐차,20


In [31]:
len_17_wrong_text_idx = []
p = re.compile('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]')
for i, one in enumerate(early_lgv1['차대번호']):
    if len(str(one)) == 17 and p.search(str(one)):
        len_17_wrong_text_idx.append(i)
len(len_17_wrong_text_idx)

0

## 저감장치 구분

In [32]:
early_lgv1['저감장치구분'].unique()

array(['조기폐차'], dtype=object)

# 12\. 사후관리 무상점검 정보

In [53]:
name = 'DLM_TB_ERP_FREE_INSPEC'
file_name = f'ODS_{name}.csv'
free_inspec = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
free_inspec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 629 entries, 0 to 628
Data columns (total 41 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   CHK_MNGT_NO             629 non-null    int64  
 1   REDUC_REG_NO            628 non-null    float64
 2   VHRNO                   629 non-null    object 
 3   VIN                     628 non-null    object 
 4   MKR_CD                  629 non-null    object 
 5   RCPT_LOCGOV_CD          628 non-null    object 
 6   PRGRE_STUS              629 non-null    object 
 7   USER_ZIP                629 non-null    object 
 8   USER_1_ADRS             629 non-null    object 
 9   USER_2_ADRS             629 non-null    object 
 10  USER_TELNO              629 non-null    object 
 11  CHK_DE                  629 non-null    int64  
 12  CHK_CST                 628 non-null    float64
 13  CHK_ENTRPS_NM           629 non-null    object 
 14  BF_DRVNG_DSTNC          628 non-null    fl

In [54]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
free_inspec.columns = [col_dict[x] for x in free_inspec.columns]
free_inspec.columns

Index(['미정', '저감등록번호', '차량등록번호', '차대번호', '미정', '미정', '무상점검진행상태', '미정', '미정',
       '미정', '미정', '점검일자', '미정', '점검업체명', '미정', '이후주행거리', '미정', '미정', '미정',
       '보조금대상YN', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정',
       '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정'],
      dtype='object')

In [55]:
free_inspec1 = free_inspec[['저감등록번호', '차량등록번호', '차대번호', '무상점검진행상태', '점검일자', '점검업체명', '이후주행거리', '보조금대상YN']]

In [56]:
free_inspec1.isnull().sum()

저감등록번호      1
차량등록번호      0
차대번호        1
무상점검진행상태    0
점검일자        0
점검업체명       0
이후주행거리      0
보조금대상YN     1
dtype: int64

In [57]:
col = pd.DataFrame(col_dict, index=['내용']).T.reset_index()
col.columns = ['컬럼명', '내용']
nul = free_inspec1.isnull().sum().reset_index()
nul.columns = ['내용', '결측 값']
col2 = col.merge(nul, on='내용', how='left')
col2

,컬럼명,내용,결측 값
0,VHRNO,차량등록번호,0.00
1,VIN,차대번호,1.00
2,SBSIDY_TRGT_YN,보조금대상YN,1.00
3,CHK_ENTRPS_NM,점검업체명,0.00
4,CHK_DE,점검일자,0.00
5,AFTR_DRVNG_DSTNC,이후주행거리,0.00
6,PRGRE_STUS,무상점검진행상태,0.00
7,REDUC_REG_NO,저감등록번호,1.00
8,CHK_MNGT_NO,미정,NaN
9,MKR_CD,미정,NaN


# 13\. 저감장치 인증등록번호

In [58]:
name = 'DLM_TB_ERP_RED_TYPE'
file_name = f'ODS_{name}.csv'
red_type = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
red_type.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1961 entries, 0 to 1960
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SEQ_NO                     1961 non-null   int64  
 1   MKR_CD                     1961 non-null   object 
 2   CRTC_MODL_NO               1961 non-null   object 
 3   RDCDVC_SE                  1961 non-null   object 
 4   RDCDVC_KND                 1291 non-null   object 
 5   REDUC_VHCTY_SE             1589 non-null   object 
 6   SBSIDY_AMT                 1961 non-null   object 
 7   SBSIDY_SLFBDN_AMT          1020 non-null   float64
 8   SBSIDY_BIZ_YR              1953 non-null   float64
 9   TDMS_SBSIDY_AMT            869 non-null    float64
 10  LWBCT_SBSIDY_AMT           1134 non-null   float64
 11  LWBCT_TDMS_SBSIDY_AMT      869 non-null    float64
 12  CRTC_MODL_KND              1309 non-null   object 
 13  PFMCRTC_RSLT_INPT_DE       669 non-null    float

In [59]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
red_type.columns = [col_dict[x] for x in red_type.columns]
red_type.columns

Index(['일련번호', '미정', '인증모델번호', '미정', '미정', '저감차종구분', '미정', '미정', '미정', '미정',
       '미정', '미정', '인증모델종류', '미정', '미정', '성능인증배기온도값', '성능인증배기온도1기준',
       '성능인증배기온도2기준', '성능인증배기압력값', '성능인증배기압력1기준', '성능인증배기압력2기준', '저감장치엔진명',
       '미정', '미정', '미정', '미정', '미정', 'PM최종저감효율', '미정', '미정', '미정', '미정'],
      dtype='object')

In [60]:
red_type1 = red_type[['일련번호', '인증모델번호', '저감차종구분', '인증모델종류', '성능인증배기온도값', 
                      '성능인증배기온도1기준', '성능인증배기온도2기준', '성능인증배기압력값', '성능인증배기압력1기준', '성능인증배기압력2기준', '저감장치엔진명', 'PM최종저감효율']]

In [61]:
red_type1.isnull().sum()

일련번호              0
인증모델번호            0
저감차종구분          372
인증모델종류          652
성능인증배기온도값      1209
성능인증배기온도1기준    1209
성능인증배기온도2기준    1209
성능인증배기압력값      1522
성능인증배기압력1기준    1522
성능인증배기압력2기준    1522
저감장치엔진명         145
PM최종저감효율        762
dtype: int64

In [62]:
col = pd.DataFrame(col_dict, index=['내용']).T.reset_index()
col.columns = ['컬럼명', '내용']
nul = red_type1.isnull().sum().reset_index()
nul.columns = ['내용', '결측 값']
col2 = col.merge(nul, on='내용', how='left')
col2

,컬럼명,내용,결측 값
0,REDUC_VHCTY_SE,저감차종구분,372.00
1,SEQ_NO,일련번호,0.00
2,RDCDVC_EGIN_NM,저감장치엔진명,145.00
3,PM_LST_REDUC_EFCNY,PM최종저감효율,762.00
4,PFMCRTC_VENTL_TMPR_VAL,성능인증배기온도값,1209.00
5,CRTC_MODL_KND,인증모델종류,652.00
6,PFMCRTC_VENTL_TMPR_1_STDR,성능인증배기온도1기준,1209.00
7,PFMCRTC_VENTL_PSSR_VAL,성능인증배기압력값,1522.00
8,PFMCRTC_VENTL_PSSR_2_STDR,성능인증배기압력2기준,1522.00
9,CRTC_MODL_NO,인증모델번호,0.00


# 14\. 저감사업 성능유지 확인검사

In [22]:
# about 10s
name = 'DLM_TB_PFM_FILE'
file_name = f'ODS_{name}.csv'
pfm = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
pfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418131 entries, 0 to 418130
Data columns (total 65 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   REDUC_REG_NO                 418117 non-null  float64
 1   PFMCFINPC_NO                 418131 non-null  object 
 2   VHRNO                        418131 non-null  object 
 3   VIN                          418131 non-null  object 
 4   PFMCFINPC_DE                 418131 non-null  int64  
 5   STRCHG_DE                    418116 non-null  float64
 6   RCPT_LOCGOV_CD               418116 non-null  object 
 7   INSPOF_CD                    418130 non-null  object 
 8   FILE_NM                      346629 non-null  object 
 9   NTJDMT_RESN                  418119 non-null  object 
 10  DRVNG_TMPR_DISTRB_JDGMNT     418058 non-null  object 
 11  PFMCFINPC_SMO_DNSTY          401461 non-null  float64
 12  PFMCFINPC_EGINPWR            415979 non-null  object 
 13 

In [23]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
pfm.columns = [col_dict[x] for x in pfm.columns]
pfm.columns

Index(['저감등록번호', '성능확인검사번호', '차량등록번호', '차대번호', '성능확인검사일자', '미정', '미정', '미정',
       '미정', '미정', '미정', '성능확인검사매연농도', '미정', '검사종류', '미정', '미정', '미정',
       '성능확인검사종합판정', '매연조건판정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정',
       '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정',
       '미정', '검사소명', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정',
       '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정',
       '미정', '미정', '미정'],
      dtype='object')

## 검사종류

In [24]:
pfm1 = pfm[['차대번호', '검사종류']]

In [25]:
pfm1.shape

(418131, 2)

In [26]:
pfm1.head()

,차대번호,검사종류
0,KL3XD70C1TK000302,부하
1,KL3YD58C1VK000409,부하
2,KL5UW63JD4P000123,부하
3,KMCCE19VPTC100083,부하
4,KMCGC17FPWC001053,부하


In [27]:
pfm1['검사종류'].unique()

array(['부하', '부하검사(KD-147)', '부하검사(LUG DOWN)', '무부하', '무부하검사(급가속)', nan],
      dtype=object)

In [28]:
pfm1.isnull().sum()

차대번호       0
검사종류    2152
dtype: int64

In [29]:
temp = pfm1.loc[pfm1['검사종류'].isnull() == True, '차대번호'].to_list()

In [32]:
tin.columns

Index(['사용연료', '주행거리', '차명', '차량번호', '차대번호', '차종', '차량연식', '엔진형식', '검사방법',
       '검사일자', '검사종류', '검사판정시각', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1',
       '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1',
       '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5',
       '무부하매연측정치6', '차량용도', '최대출력허용치', '최대출력측정값', '검사접수번호', '검사소명', '재검사기간',
       '정기정밀', '검사 검사소코드', '검사유효기간'],
      dtype='object')

In [33]:
tin['검사방법'].unique()

array(['부하검사(ASM-Idling)', '부하검사(KD-147)', '무부하검사(급가속)', '부하검사(LUG DOWN)',
       '무부하검사(TSI)', '무부하검사(정지가동)', '부하검사(ASM2525)'], dtype=object)

In [38]:
# about 42m 8s
# count : 1514
count = 0
for one in temp:
    tp = tin.loc[tin['차대번호'] == one]
    if tp.shape[0] != 0 :
        count += 1
count

1514

In [37]:
n = 0
for one in temp:
    tp = tin.loc[tin['차대번호'] == one]
    if tp.shape[0] != 0 and n <= 5:
        display(tp[['차대번호', '검사방법']])
        n += 1
    elif n > 5:
        break

,차대번호,검사방법
12563769,KMFLA19AP4C050122,부하검사(LUG DOWN)


,차대번호,검사방법
7017834,KMFWVH7HP2U474391,무부하검사(정지가동)


,차대번호,검사방법
7170848,44-99-343-94818,부하검사(LUG DOWN)


,차대번호,검사방법
21762234,KL3PD62C1YK000028,무부하검사(급가속)


,차대번호,검사방법
21221131,KMFXKS7BPWU184963,부하검사(KD-147)


,차대번호,검사방법
7035364,KMFLA19RPVC117945,무부하검사(급가속)


# 15\. 운행제한 발령정보

In [63]:
# about 0s
name = 'N_IS_ISSUE'
file_name = f'ODS_{name}.csv'
issue = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
issue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1785 entries, 0 to 1784
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   GNFD_NO             1785 non-null   int64  
 1   RPST_GNFD_NO        1785 non-null   int64  
 2   DNSTY_STDR_ID       1785 non-null   object 
 3   TY_STDR_ID          1785 non-null   object 
 4   GNFD_NM             1785 non-null   object 
 5   GNFD_EXEC_CND_FLAG  1785 non-null   object 
 6   GNFD_EXEC_RESRV_DT  1784 non-null   object 
 7   BEFT_ESTBS_ID       0 non-null      float64
 8   DC                  1785 non-null   object 
 9   GNFD_EXC_FLAG       1785 non-null   object 
 10  HMPG_INDIC_YN       1785 non-null   object 
 11  LCINFO_API_YN       1785 non-null   object 
 12  REG_ID              1785 non-null   object 
 13  REG_DT              1785 non-null   object 
 14  UPDT_ID             1652 non-null   object 
 15  UPDT_DT             1652 non-null   object 
dtypes: flo

In [64]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
issue.columns = [col_dict[x] for x in issue.columns]
issue.columns

Index(['발령 번호', '미정', '농도기준 발령구분', '유형 구분', '미정', '미정', '미정', '미정', '미정', '미정',
       '미정', '미정', '미정', '미정', '미정', '수정 일시'],
      dtype='object')

In [65]:
issue1 = issue[['유형 구분', '농도기준 발령구분', '발령 번호']]

In [66]:
issue1.isnull().sum()

유형 구분        0
농도기준 발령구분    0
발령 번호        0
dtype: int64

## 유형 구분

In [67]:
issue1['유형 구분'].unique()

array(['T002', 'T001'], dtype=object)

## 농도기준 발령구분

In [68]:
issue1['농도기준 발령구분'].unique()

array(['C012', 'C011'], dtype=object)

## 발령 번호

In [69]:
issue1['발령 번호'].unique()

array([2142, 2143, 2227, ..., 2139, 2140, 2141], dtype=int64)

In [70]:
issue1['발령 번호'].describe()

count   1785.00
mean    1552.90
std      691.88
min        1.00
25%     1020.00
50%     1525.00
75%     2171.00
max     2653.00
Name: 발령 번호, dtype: float64

In [72]:
issue1[issue1['발령 번호'] < 1000].sort_values('발령 번호').head()

,유형 구분,농도기준 발령구분,발령 번호
1534,T001,C011,1
1536,T001,C011,21
1537,T001,C012,41
1538,T001,C012,42
1535,T001,C012,43


# 16\. 비상시 및 계절제 단속발령

In [74]:
# about 1m 35s
name = 'N_IS_ISSUE_DISCLOSURE'
file_name = f'ODS_{name}_10.txt'
issue_dis = pd.read_table(os.path.join(issue_disclosure_fold, file_name), low_memory=False)
issue_dis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5727752 entries, 0 to 5727751
Data columns (total 25 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   REGLT_NO            int64 
 1   GNFD_NO             int64 
 2   REG_SIDO_CD         int64 
 3   REG_SIGNGU_CD       int64 
 4   REGLT_AREA_CD       int64 
 5   TFCLNE_ID           object
 6   DISCL_DEHMS         int64 
 7   VHMNO               object
 8   VIN                 object
 9   VHRNO               object
 10  FILE_NM             object
 11  FILE_CURS_NM        object
 12  ORG_FILE_CURS_NM    object
 13  THUMB_FILE_CURS_NM  object
 14  DTL_FILE_CURS_NM    object
 15  REG_ID              object
 16  REG_DT              object
 17  EXCLU_REG_DT        object
 18  RDCDVC_YN           object
 19  BSN_YN              object
 20  RWMR_YN             object
 21  DSPN_YN             object
 22  RQT_YN              object
 23  UNDEVT_YN           object
 24  SND_YN              object
dtypes: int64(6), objec

In [76]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
issue_dis.columns = [col_dict[x] for x in issue_dis.columns]
issue_dis.columns

Index(['미정', '발령 번호', '미정', '미정', '운행제한 단속 발령대상 지역코드', '미정', '적발 일자시각',
       '차량관리번호', '차대번호', '차량등록번호', '미정', '미정', '미정', '미정', '미정', '미정', '미정',
       '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정'],
      dtype='object')

In [77]:
issue_dis1 = issue_dis[['차량관리번호', '차대번호', '차량등록번호', '발령 번호']]

## 차량관리번호

In [78]:
issue_dis1['차량관리번호'].unique()

array(['KMCVK17PP6C035326-01', 'KMHSJ81WP6U005551-01',
       'KMHMH81VP7U302554-01', ..., 'KMFLA19RPYC011042-01',
       'KPAKA4AD15P386365-01', 'KNAJC52183A087705-01'], dtype=object)

In [86]:
wrong_text_idx = []
p = re.compile('[A-Z0-9][-][0-9]')
for i, one in enumerate(issue_dis1['차량관리번호']):
    if not p.search(str(one)):
        wrong_text_idx.append(i)
len(wrong_text_idx)

16146

In [87]:
issue_dis1.loc[wrong_text_idx, '차량관리번호'].head()

499     4799경북86아2461
501       119990러4522
521       489997너9624
1089    4599전북83사1074
1231    4199경기93자2547
Name: 차량관리번호, dtype: object

In [88]:
issue_dis1.loc[wrong_text_idx, '차량관리번호'].tail()

5725665      119990머4206
5726202    2899인천98사2015
5726267    2699부산90아2280
5726427    4199경기71바7387
5727607    2699부산90바3923
Name: 차량관리번호, dtype: object

## 차대번호

In [79]:
vin_temp = issue_dis1['차대번호'].str.len()
set(vin_temp.unique())

{9, 11, 12, 13, 14, 15, 16, 17}

In [80]:
count = 0
for one in set(vin_temp.unique()):
    temp = issue_dis1.loc[issue_dis1['차대번호'].str.len() == one, '차대번호']
    count += temp.shape[0]
    print(f'차대번호 길이 : {one}')
    print(temp.shape)
    display(temp.head())
    print()
count

차대번호 길이 : 9
(8,)


159194     CN67C4394
340630     CN67C4394
551553     CN67C4394
1404140    CN67C4394
4307042    CN67C4394
Name: 차대번호, dtype: object


차대번호 길이 : 11
(393,)


108003    01891200094
112162    08804360250
133678    KN67C003987
160737    SL44C011395
172081    01861321189
Name: 차대번호, dtype: object


차대번호 길이 : 12
(16366,)


...


차대번호 길이 : 16
(63,)


42204     42-94-322-00002태
109783    KN3HD3353K946676
277958    44-94-343-09254공
385918    48-00-342-35197창
630131    44-94-343-09254공
Name: 차대번호, dtype: object


차대번호 길이 : 17
(5710433,)


0    KMCVK17PP6C035326
1    KMHSJ81WP6U005551
2    KMHMH81VP7U302554
3    YV2JS02C17A642968
4    KMFLA18EP7C024211
Name: 차대번호, dtype: object

5727752

In [81]:
len_17_wrong_text_idx = []
p = re.compile('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]')
for i, one in enumerate(issue_dis1['차대번호']):
    if len(str(one)) == 17 and p.search(str(one)):
        len_17_wrong_text_idx.append(i)
len(len_17_wrong_text_idx)

0

In [82]:
issue_dis1.loc[len_17_wrong_text_idx, '차대번호'].head()

Series([], Name: 차대번호, dtype: object)

## 차량등록번호

In [83]:
issue_dis1['차량등록번호'].unique()

array(['92보8509', '37나2483', '43저9028', ..., '87루9080', '89가6066',
       '05다1127'], dtype=object)

In [84]:
issue_dis1['차량등록번호'].isnull().sum()

0

In [85]:
p1 = re.compile('[가-힣]{2}[0-9]{1,2}[가-힣]{1}[0-9]{4}')
p2 = re.compile('[0-9]{2,3}[가-힣]{1}[0-9]{4}')
wrong_vhrno_list = []
for one in tqdm(issue_dis1['차량등록번호']):
    if not p1.search(str(one)) and not p2.search(str(one)):
        wrong_vhrno_list.append(one)
len(wrong_vhrno_list)

100%|██████████| 5727752/5727752 [00:07<00:00, 797652.57it/s]


0

## 발령 번호

In [91]:
issue_dis1['발령 번호'].describe()

count   5727752.00
mean       2506.78
std         109.98
min        2342.00
25%        2411.00
50%        2438.00
75%        2614.00
max        2666.00
Name: 발령 번호, dtype: float64

# 17\. 운행제한 단속정보

In [5]:
# about 
name = 'N_US_DISCLOSURE'
file_name = f'ODS_{name}.csv'
ld_ckeck = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
ld_ckeck.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3090035 entries, 0 to 3090034
Data columns (total 17 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   NO                  int64 
 1   ODR_YM              int64 
 2   REG_SIDO_CD         int64 
 3   REG_SIGNGU_CD       int64 
 4   DISCL_TY            object
 5   REGLT_AREA_CD       int64 
 6   TFCLNE_ID           object
 7   DISCL_DEHMS         int64 
 8   VHMNO               object
 9   VIN                 object
 10  VHRNO               object
 11  FILE_NM             object
 12  ORG_FILE_CURS_NM    object
 13  THUMB_FILE_CURS_NM  object
 14  DTL_FILE_CURS_NM    object
 15  REG_ID              object
 16  REG_DT              object
dtypes: int64(6), object(11)
memory usage: 400.8+ MB


In [6]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
ld_ckeck.columns = [col_dict[x] for x in ld_ckeck.columns]
ld_ckeck.columns

Index(['미정', '미정', '미정', '미정', '적발유형', '단속지역코드', '미정', '적발일시', '차량관리번호',
       '차대번호', '차량등록번호', '미정', '미정', '미정', '미정', '미정', '미정'],
      dtype='object')

In [8]:
ld_ckeck = ld_ckeck[['차대번호', '차량등록번호', '적발유형', '차량관리번호']]

In [9]:
ld_ckeck.shape

(3090035, 4)

## 차대번호

In [10]:
ld_ckeck['차대번호'].isnull().sum()

0

In [11]:
vin_temp = ld_ckeck['차대번호'].str.len()
set(vin_temp.unique())

{9, 11, 12, 13, 15, 16, 17}

In [12]:
count = 0
for one in set(vin_temp.unique()):
    temp = ld_ckeck.loc[ld_ckeck['차대번호'].str.len() == one, '차대번호']
    count += temp.shape[0]
    print(f'차대번호 길이 : {one}')
    print(temp.shape)
    display(temp.head())
    print()
count

차대번호 길이 : 9
(2,)


1104414    CN67C4394
1325228    CN67C4394
Name: 차대번호, dtype: object


차대번호 길이 : 11
(805,)


158720    10771222554
256294    10814246954
269095    10814246954
461701    10792126524
545140    13831219384
Name: 차대번호, dtype: object


차대번호 길이 : 12
(4857,)


...


차대번호 길이 : 16
(32,)


623329    YV2J4CF34A585755
761206    YV2J4CF34A585755
761818    YV2J4CF34A585755
822010    YV2J4CF34A585755
822765    YV2J4CF34A585755
Name: 차대번호, dtype: object


차대번호 길이 : 17
(3084029,)


0    KPBFA2AD12P021974
1    KMHNN81XP2U038507
2    KMFZCD7JP5U094049
3    KMFWVH7HP4U579325
4    KNAMA75433S411400
Name: 차대번호, dtype: object

3090035

In [13]:
len_17_wrong_text_idx = []
p = re.compile('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]')
for i, one in enumerate(ld_ckeck['차대번호']):
    if len(str(one)) == 17 and p.search(str(one)):
        len_17_wrong_text_idx.append(i)
len(len_17_wrong_text_idx)

0

In [14]:
ld_ckeck.loc[len_17_wrong_text_idx, '차대번호'].head()

Series([], Name: 차대번호, dtype: object)

## 차량등록번호

In [15]:
ld_ckeck['차량등록번호'].unique()

array(['29다5775', '14도9561', '80루1945', ..., '12도4921', '80고2805',
       '59버4907'], dtype=object)

In [16]:
ld_ckeck['차량등록번호'].isnull().sum()

0

In [17]:
p1 = re.compile('[가-힣]{2}[0-9]{1,2}[가-힣]{1}[0-9]{4}')
p2 = re.compile('[0-9]{2,3}[가-힣]{1}[0-9]{4}')
wrong_vhrno_list = []
for one in tqdm(ld_ckeck['차량등록번호']):
    if not p1.search(str(one)) and not p2.search(str(one)):
        wrong_vhrno_list.append(one)
len(wrong_vhrno_list)

100%|██████████| 3090035/3090035 [00:03<00:00, 909274.58it/s]


0

## 적발유형

In [18]:
ld_ckeck['적발유형'].unique()

array(['S'], dtype=object)

In [19]:
ld_ckeck['적발유형'].isnull().sum()

0

## 차량관리번호

In [20]:
ld_ckeck['차량관리번호'].unique()

array(['KPBFA2AD12P021974-01', 'KMHNN81XP2U038507-01',
       'KMFZCD7JP5U094049-01', ..., 'KNAJC52183A118702-01',
       'KN3HNP6D34K015249-01', 'KMHSD81VP5U008579-01'], dtype=object)

In [22]:
ld_ckeck['차량관리번호'].isnull().sum()

0

In [23]:
len('KPBFA2AD12P021974')

17

In [24]:
wrong_text_idx = []
p = re.compile('[A-Z0-9][-][0-9]')
for i, one in enumerate(ld_ckeck['차량관리번호']):
    if not p.search(str(one)):
        wrong_text_idx.append(i)
len(wrong_text_idx)

13824

In [25]:
ld_ckeck.loc[wrong_text_idx, '차량관리번호'].head()

721     1199서울85바9660
753     1199서울85바9660
918       289987가5059
1026    1199서울85바9660
1196    1199서울85바9660
Name: 차량관리번호, dtype: object

In [26]:
ld_ckeck.loc[wrong_text_idx, '차량관리번호'].tail()

3088277    4199경기89아2105
3089107    4199경기89아2105
3089185    4199경기89아2105
3089646    4199경기89아2105
3089961    4199경기89아2105
Name: 차량관리번호, dtype: object

# 18\. 정밀검사 검사정보

In [ ]:
# about 11m 14s
name = 'TB_EET_HIS_ME'
file_name = f'ODS_{name}.csv'
his_me = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
his_me.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28682228 entries, 0 to 28682227
Data columns (total 75 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   SMO_CHK_ORG_NM         object 
 1   SMO_CHK_DATE           int64  
 2   SMO_CHK_TYPE           object 
 3   SMO_CHK_REG_NO         object 
 4   CAR_REG_NO             object 
 5   CAR_SER_NO             object 
 6   CAR_SPEC_REG_NO        object 
 7   CAR_NM                 object 
 8   CAR_TYPE_NM            object 
 9   USAGE                  object 
 10  CAR_YEAR               int64  
 11  CAR_FUEL_NM            object 
 12  CAR_MILE               float64
 13  CAR_LENGTH             float64
 14  CAR_WIDTH              float64
 15  CAR_HEIGHT             float64
 16  ENGINE_TYPE            object 
 17  ENGINE_CAPACITY        float64
 18  ENGINE_POWER           object 
 19  CAR_WEIGHT             float64
 20  TOT_WEIGHT             float64
 21  CAR_PASSENGER          float64
 22  CONT            

In [ ]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
his_me.columns = [col_dict[x] for x in his_me.columns]
his_me.columns

Index(['정밀검사소명', '정밀검사일자', '정밀검사종류', '정밀검사접수번호', '차량번호', '차대번호', '미정', '차명',
       '차종', '차량용도', '차량연식', '사용연료', '주행거리', '미정', '미정', '미정', '엔진형식', '미정',
       '미정', '미정', '미정', '미정', '미정', '미정', '정밀검사유효기간', '미정', '미정', '정밀검사측정방법',
       '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2',
       '무부하매연판정2', '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4',
       '무부하매연허용치4', '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '이산화탄소값',
       '최대출력허용치', '최대출력측정값', '산소값', '미정', '미정', '정밀검사판정', '재검사기간', '미정', '미정',
       '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '정밀검사시각정보',
       '정밀검사 검사소코드', '미정', '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6', '미정', '미정',
       '미정', '미정', '미정', '미정', '미정', '미정'],
      dtype='object')

In [ ]:
his_me['사용연료'].unique()

array(['경유', '휘발유', '휘발유(무연)', '엘피지', '하이브리드(휘발유+전기)', '하이브리드(LPG+전기)',
       '휘발유(유연)', '하이브리드(경유+전기)', '기타연료', 'LNG', 'CNG', nan], dtype=object)

In [6]:
# about 5m 48s
name = '[ODS]정밀검사(2022.12.28)'
file_name = f'{name}.csv'
dins = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
dins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28682228 entries, 0 to 28682227
Data columns (total 41 columns):
 #   Column     Dtype  
---  ------     -----  
 0   사용연료       object 
 1   주행거리       float64
 2   차명         object 
 3   차량번호       object 
 4   차대번호       object 
 5   차종         object 
 6   차량연식       int64  
 7   엔진형식       object 
 8   검사방법       object 
 9   검사일자       int64  
 10  검사종류       object 
 11  검사판정시각     object 
 12  검사판정       object 
 13  산소값        float64
 14  이산화탄소값     float64
 15  무부하매연판정1   object 
 16  무부하매연판정2   object 
 17  무부하매연판정3   object 
 18  무부하매연판정4   object 
 19  무부하매연판정5   object 
 20  무부하매연판정6   object 
 21  무부하매연허용치1  float64
 22  무부하매연허용치2  float64
 23  무부하매연허용치3  float64
 24  무부하매연허용치4  float64
 25  무부하매연허용치5  float64
 26  무부하매연허용치6  float64
 27  무부하매연측정치1  float64
 28  무부하매연측정치2  float64
 29  무부하매연측정치3  float64
 30  무부하매연측정치4  float64
 31  무부하매연측정치5  float64
 32  무부하매연측정치6  float64
 33  차량용도       object 
 34  최대출력허용치    float

In [7]:
dins.columns

Index(['사용연료', '주행거리', '차명', '차량번호', '차대번호', '차종', '차량연식', '엔진형식', '검사방법',
       '검사일자', '검사종류', '검사판정시각', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1',
       '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1',
       '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5',
       '무부하매연측정치6', '차량용도', '최대출력허용치', '최대출력측정값', '검사접수번호', '검사소명', '재검사기간',
       '검사 검사소코드', '검사유효기간'],
      dtype='object')

## 사용연료

In [12]:
dins['사용연료'].unique()

array(['경유', '휘발유', '휘발유(무연)', '엘피지', '하이브리드(휘발유+전기)', '하이브리드(LPG+전기)',
       '휘발유(유연)', '하이브리드(경유+전기)', '기타연료', 'LNG', 'CNG', nan], dtype=object)

In [13]:
dins.loc[dins['사용연료'].isnull() == True, ['사용연료', '차명']].head()

,사용연료,차명
16129609,NaN,SM3


In [8]:
dins.loc[dins['사용연료'].isnull() == True, ['사용연료', '차명']].iloc[50:60]

,사용연료,차명


In [25]:
dins.loc[dins['사용연료'].isnull() == True, ['사용연료', '차대번호', '엔진형식']]

,사용연료,차대번호,엔진형식
16129609,NaN,KNMC4C2HMDP139016,H4MD


In [20]:
ins.loc[ins['차대번호'] == 'KNMC4C2HMDP139016', ['차대번호', '사용연료']]

,차대번호,사용연료


In [22]:
tin.loc[tin['차대번호'] == 'KNMC4C2HMDP139016', ['차대번호', '사용연료']]

,차대번호,사용연료
15699395,KNMC4C2HMDP139016,NaN


In [23]:
tdf.loc[tdf['차대번호'] == 'KNMC4C2HMDP139016', ['차대번호', '제원관리번호', '연료']]

,차대번호,제원관리번호,연료
12605884,KNMC4C2HMDP139016,A0410000600091212,휘발유


In [26]:
tdf.loc[tdf['엔진형식'] == 'H4MD', ['차대번호', '제원관리번호', '연료', '엔진형식']]

,차대번호,제원관리번호,연료,엔진형식
11737503,KNMC4C2HMDP129948,A0410000600091212,휘발유,H4MD
11738257,KNMC4C2HMDP129937,A0410000600091212,휘발유,H4MD
11738347,KNMC4C2HMDP130344,A0410000600091212,휘발유,H4MD
11738355,KNMC4C2HMDP129938,A0410000600091212,휘발유,H4MD
11739835,KNMC4C2HMDP129929,A0410000600091212,휘발유,H4MD
...,...,...,...,...
28722354,KNMC4C2HMEP151205,99934125271161222,휘발유,H4MD
28816489,KNMC4C2HMFP173142,99934125271861222,휘발유,H4MD
29021910,KNMC4C2HMFP171689,9993411b112341222,휘발유,H4MD
29206036,KNMC4C2HMGP198834,9993411b117651222,휘발유,H4MD


## 차명

In [9]:
dins['차명'].isnull().sum()

0

## 차량번호

In [10]:
dins['차량번호'].isnull().sum()

0

In [11]:
dins['차량번호'].unique()

array(['90거4483', '58오7606', '35로2372', ..., '62조5474', '58다1021',
       '61너7414'], dtype=object)

In [12]:
p1 = re.compile('[가-힣]{2}[0-9]{1,2}[가-힣]{1}[0-9]{4}')
p2 = re.compile('[0-9]{2,3}[가-힣]{1}[0-9]{4}')
wrong_vhrno_list = []
for one in tqdm(dins['차량번호']):
    if not p1.search(str(one)) and not p2.search(str(one)):
        wrong_vhrno_list.append(one)
len(wrong_vhrno_list)

100%|██████████| 28682228/28682228 [00:35<00:00, 801902.14it/s]


204

In [13]:
wrong_vhrno_list[:5]

['#6부2822', '#6부2822', '#6부2822', '#6부2822', '#6가8769']

In [14]:
wrong_vhrno_list[-5:]

['#0거0682', '#0거0682', '#0거0682', '#0거0682', '#0거0682']

## 차대번호

In [15]:
vin_temp = dins['차대번호'].str.len()
set(vin_temp.unique())

{10, 11, 12, 13, 14, 15, 16, 17}

In [16]:
count = 0
for one in set(vin_temp.unique()):
    temp = dins.loc[dins['차대번호'].str.len() == one, '차대번호']
    count += temp.shape[0]
    print(f'차대번호 길이 : {one}')
    print(temp.shape)
    display(temp.head())
    print()
count

차대번호 길이 : 10
(3,)


7399569     1162127346
22917565    1162127346
26691669    1332942254
Name: 차대번호, dtype: object


차대번호 길이 : 11
(495,)


149419    01901321587
171339    01911322533
223673    01894100061
236602    03904300818
283596    01901321837
Name: 차대번호, dtype: object


차대번호 길이 : 12
(12076,)


...


차대번호 길이 : 16
(283,)


504      KNAFH22139A00852
12842    VF36D3FYF8L00078
29941    KMHSD81VP3U43335
41691    KMHSD81VP3U43335
93137    KMHDN41BP5U61314
Name: 차대번호, dtype: object


차대번호 길이 : 17
(28668916,)


0    KNCSJY76BDK832102
1    KMHFB41BBBA559634
2    KMHGB41AP9U047697
3    KNAPC813BBK099365
4    KMHFF41EBBA002301
Name: 차대번호, dtype: object

28682228

In [17]:
len_17_wrong_text_idx = []
p = re.compile('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]')
for i, one in enumerate(dins['차대번호']):
    if len(str(one)) == 17 and p.search(str(one)):
        len_17_wrong_text_idx.append(i)
len(len_17_wrong_text_idx)

0

In [18]:
dins.loc[len_17_wrong_text_idx, '차대번호'].head()

Series([], Name: 차대번호, dtype: object)

## 차종

In [19]:
dins['차종'].isnull().sum()

0

In [20]:
dins['차종'].unique()

array(['화물차일반형소형', '승용차일반형대형', '승용차다목적형중형', '화물차일반형중형', '승합차일반형중형',
       '승용차일반형경형', '승용차다목적형대형', '승용차일반형소형', '승용차일반형중형', '화물차밴형소형',
       '승합차일반형경형', '화물차일반형대형', '화물차덤프형소형', '승용차기타형중형', '화물차특수용도형소형',
       '특수차특수작업형소형', '화물차특수용도형중형', '화물차특수용도형대형', '특수차특수작업형중형', '승용차기타형대형',
       '화물차밴형중형', '승용차겸화물형경형', '승합차일반형대형', '승합차특수형중형', '화물차일반형경형',
       '화물차밴형경형', '특수차견인형대형', '화물차덤프형중형', '특수차구난형중형', '특수차구난형소형',
       '특수차특수작업형대형', '화물차덤프형대형', '화물차탱크로리형대형', '화물차특수용도형경형', '특수차구난형대형',
       '화물차탱크로리형중형', '승용차겸화물형대형', '승용차다목적형경형', '승합차특수형대형', '승용차기타형경형',
       '승용차다목적형소형', '특수차견인형소형', '승합차일반형소형', '승용차겸화물형중형', '승용차기타형소형',
       '특수차특수작업형경형', '승합차특수형소형', '특수차견인형중형', '승합차특수형경형', '화물차덤프형경형',
       '화물차탱크로리형경형', '승용차겸화물형소형', '화물차밴형대형', '화물차탱크로리형소형', '특수차특수용도형대형',
       '특수차특수용도형중형', '특수차특수용도형소형', '특수차특수용도형경형'], dtype=object)

In [ ]:
# '승용차일반형경형',
# '승용차일반형소형',
# '승용차일반형중형',
# '승용차일반형대형',

# '승용차겸화물형경형',
# '승용차겸화물형소형',
# '승용차겸화물형중형',
# '승용차겸화물형대형',

# '승용차다목적형경형',
# '승용차다목적형소형',
# '승용차다목적형중형',
# '승용차다목적형대형',

# '승용차기타형경형',
# '승용차기타형소형',
# '승용차기타형중형',
# '승용차기타형대형',

In [ ]:
# '승합차일반형경형',
# '승합차일반형소형',
# '승합차일반형중형',
# '승합차일반형대형',

# '승합차특수형경형',
# '승합차특수형소형',
# '승합차특수형중형',
# '승합차특수형대형',

In [ ]:
# '화물차일반형경형',
# '화물차일반형소형',
# '화물차일반형중형',
# '화물차일반형대형',

# '화물차밴형경형',
# '화물차밴형소형',
# '화물차밴형중형',
# '화물차밴형대형',

# '화물차덤프형경형',
# '화물차덤프형소형',
# '화물차덤프형중형',
# '화물차덤프형대형',

# '화물차탱크로리형경형',
# '화물차탱크로리형소형',
# '화물차탱크로리형중형',
# '화물차탱크로리형대형',

# '화물차피견인형저상형',
# '화물차피견인형고상형',
# '화물차피견인형샷시형',
# '화물차피견인형적재함형',
# '화물차피견인형기타형',

# '화물차특수용도형경형',
# '화물차특수용도형소형',
# '화물차특수용도형중형',
# '화물차특수용도형대형',

In [ ]:
# '특수차특수작업형경형',
# '특수차특수작업형소형',
# '특수차특수작업형중형',
# '특수차특수작업형대형',

# '특수차구난형소형',
# '특수차구난형중형',
# '특수차구난형대형',

# '특수차견인형소형',
# '특수차견인형중형',
# '특수차견인형대형',

# '특수차특수용도형경형',
# '특수차특수용도형소형',
# '특수차특수용도형중형',
# '특수차특수용도형대형',

## 차량연식

In [21]:
dins['차량연식'].isnull().sum()

0

In [22]:
dins['차량연식'].describe()

count   28682228.00
mean        2011.18
std            4.54
min         1936.00
25%         2008.00
50%         2012.00
75%         2015.00
max         2022.00
Name: 차량연식, dtype: float64

In [23]:
dins['차량연식'].head()

0    2013
1    2011
2    2009
3    2011
4    2011
Name: 차량연식, dtype: int64

In [24]:
count = 0
for one in dins['차량연식']:
    if len(str(one)) != 4:
        count += 1
count

0

## 엔진형식

In [27]:
dins['엔진형식'].isnull().sum()

382

In [28]:
dins.loc[dins['엔진형식'].isnull() == True, ['엔진형식', '차대번호']].head()

,엔진형식,차대번호
152905,NaN,KN90J5MFDAHZ00006
310843,NaN,KLACM19F1JB044969
545422,NaN,KN3SAP8Y14K021785
646382,NaN,JTJHF10UX10187515
653870,NaN,KMHWF35V3YA287902


In [29]:
tdf.loc[tdf['차대번호'] == 'KN90J5MFDAHZ00006', ['차대번호', '제원관리번호', '엔진형식']]

,차대번호,제원관리번호,엔진형식
8708926,KN90J5MFDAHZ00006,A0810005008583209,D4GA


## 정밀검사일자

In [27]:
dins.columns

Index(['사용연료', '주행거리', '차명', '차량번호', '차대번호', '차종', '차량연식', '엔진형식', '검사방법',
       '검사일자', '검사종류', '검사판정시각', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1',
       '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1',
       '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5',
       '무부하매연측정치6', '차량용도', '최대출력허용치', '최대출력측정값', '검사접수번호', '검사소명', '재검사기간',
       '검사 검사소코드', '검사유효기간'],
      dtype='object')

In [28]:
dins['검사일자'].isnull().sum()

0

In [29]:
dins['검사일자'].unique()

array([20190102, 20190103, 20190104, ..., 20230105, 20230106, 20230107],
      dtype=int64)

In [30]:
count = 0
for one in dins['검사일자']:
    if len(str(one)) != 8:
        count += 1
count

0

## 정밀검사측정방법

In [31]:
dins['검사방법'].isnull().sum()

0

In [144]:
dins['검사방법'].unique()

array(['부하검사(KD-147)', '부하검사(ASM-Idling)', '무부하검사(급가속)', '부하검사(LUG DOWN)',
       '무부하검사(TSI)', '무부하검사(정지가동)', '부하검사(ASM2525)'], dtype=object)

## 정밀검사종류

In [33]:
dins['검사종류'].isnull().sum()

0

In [34]:
dins['검사종류'].unique()

array(['종합검사', '2부재검사(종합)', '종합검사(경과)', '1부재검사(종합)'], dtype=object)

# 19\. 저감장치 부착AS

In [63]:
name = 'TB_ERP_ATT_AS'
file_name = f'ODS_{name}.csv'
att_as = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
att_as.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20838 entries, 0 to 20837
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ERP_REG_NO       20838 non-null  int64  
 1   AS_DATE          20838 non-null  int64  
 2   AS_CONT          20838 non-null  object 
 3   AS_REASON        20461 non-null  object 
 4   AS_MONEY         20838 non-null  int64  
 5   AS_MONEY_METHOD  19037 non-null  object 
 6   CAR_REG_NO       20838 non-null  object 
 7   CAR_SER_NO       20833 non-null  object 
 8   ATT_DATE         19454 non-null  float64
 9   MAKER_ORG_CD     20838 non-null  object 
 10  CONF_CHG_DATE    7960 non-null   float64
 11  ETC_CONT1        20838 non-null  int64  
 12  ETC_CONT2        0 non-null      float64
 13  ETC_CONT3        0 non-null      float64
 14  REC_DATE         0 non-null      float64
 15  REC_NM           0 non-null      float64
dtypes: float64(6), int64(4), object(6)
memory usage: 2.5+ MB


In [64]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
att_as.columns = [col_dict[x] for x in att_as.columns]
att_as.columns

Index(['저감등록번호', 'AS 일자', 'AS 횟수', 'AS 사유', '미정', '미정', '미정', '차대번호', '미정',
       '미정', '미정', '미정', '미정', '미정', '미정', '미정'],
      dtype='object')

In [65]:
att_as1 = att_as[['저감등록번호', 'AS 일자', 'AS 횟수', 'AS 사유', '차대번호']]

In [66]:
att_as1.isnull().sum()

저감등록번호      0
AS 일자       0
AS 횟수       0
AS 사유     377
차대번호        5
dtype: int64

In [67]:
col = pd.DataFrame(col_dict, index=['내용']).T.reset_index()
col.columns = ['컬럼명', '내용']
nul = att_as1.isnull().sum().reset_index()
nul.columns = ['내용', '결측 값']
col2 = col.merge(nul, on='내용', how='left')
col2

,컬럼명,내용,결측 값
0,CAR_SER_NO,차대번호,5.00
1,ERP_REG_NO,저감등록번호,0.00
2,AS_REASON,AS 사유,377.00
3,AS_DATE,AS 일자,0.00
4,AS_CONT,AS 횟수,0.00
5,AS_MONEY,미정,NaN
6,AS_MONEY_METHOD,미정,NaN
7,CAR_REG_NO,미정,NaN
8,ATT_DATE,미정,NaN
9,MAKER_ORG_CD,미정,NaN


# 20\. 저감장치 부착이력

In [93]:
# about 17s
name = 'TB_ERP_ATT_HIS'
file_name = f'ODS_{name}.csv'
att = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 810777 entries, 0 to 810776
Columns: 103 entries, ERP_REG_NO to THREE_WAY_CATALYST_RETURN_YN
dtypes: float64(32), int64(3), object(68)
memory usage: 637.1+ MB


In [94]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
att.columns = [col_dict[x] for x in att.columns]
att.columns

Index(['저감등록번호', '미정', '미정', '차량번호', '미정', '차대번호', '미정', '미정', '미정', '미정',
       ...
       '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '삼원촉매장치반납여부'],
      dtype='object', length=103)

In [95]:
att = att[['저감장치종류', '차량번호', '차대번호', '저감장치구분']]

In [96]:
att.isnull().sum()

저감장치종류    223879
차량번호           0
차대번호           0
저감장치구분         0
dtype: int64

## 저감장치종류

In [97]:
att['저감장치종류'].unique()

array([nan, 'DOC', '자연대형', '자연중형', '복합중형', 'pDPF', '복합대형', '중형', '복합소형',
       '소형', '자연대형+SCR', '대형'], dtype=object)

## 차량번호

In [98]:
att['차량번호'].unique()

array(['경기81라6983', '서울83고6732', '인천83가2144', ..., '52구6335', '02모2923',
       '28주7293'], dtype=object)

In [99]:
p1 = re.compile('[가-힣]{2}[0-9]{1,2}[가-힣]{1}[0-9]{4}')
p2 = re.compile('[0-9]{2,3}[가-힣]{1}[0-9]{4}')
wrong_vhrno_list = []
for one in tqdm(att['차량번호']):
    if not p1.search(str(one)) and not p2.search(str(one)):
        wrong_vhrno_list.append(one)
len(wrong_vhrno_list)

100%|██████████| 810777/810777 [00:00<00:00, 887545.84it/s]


178

In [100]:
wrong_vhrno_list[:5]

['28-4221', '14-1887', '경기사4859', '서울04-9105', '서울15-8916']

In [101]:
wrong_vhrno_list[-5:]

['제주04-1524', '강원04-3821', '강원04-4056', '제주04-1350', '제주04-2467']

## 차대번호

In [102]:
vin_temp = att['차대번호'].str.len()
set(vin_temp.unique())

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}

In [104]:
count = 0
for one in set(vin_temp.unique()):
    temp = att.loc[att['차대번호'].str.len() == one, '차대번호']
    count += temp.shape[0]
    print(f'차대번호 길이 : {one}')
    print(temp.shape)
    display(temp.head())
    print()
count

차대번호 길이 : 1
(4,)


431711    0
431713    0
431714    0
431715    0
Name: 차대번호, dtype: object


차대번호 길이 : 2
(1,)


701124    78
Name: 차대번호, dtype: object


차대번호 길이 : 3
(3,)


...


차대번호 길이 : 19
(109,)


438213    KMJFD3737BPYU462313
763682    CMP450D-1067-6877KF
763734    CMP570D-0651-6882KF
764362    CMP230D-2817-6874KF
764958    CMP230D-4244-6874KF
Name: 차대번호, dtype: object


차대번호 길이 : 20
(5,)


774983    FX70D-FA50DT60400022
776283    PF3DPAHLTHFRHDDPDF13
791827    CMMP230D-1182-6874KF
798169    FX50D-FA50DT60400025
798481    FX70D-FA50T-60900014
Name: 차대번호, dtype: object

810777

In [105]:
len_17_wrong_text_idx = []
p = re.compile('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]')
for i, one in enumerate(att['차대번호']):
    if len(str(one)) == 17 and p.search(str(one)):
        len_17_wrong_text_idx.append(i)
len(len_17_wrong_text_idx)

0

In [106]:
att.loc[len_17_wrong_text_idx, '차대번호'].head()

Series([], Name: 차대번호, dtype: object)

## 저감장치구분

In [107]:
att['저감장치구분'].unique()

array(['엔진개조', '3종', '1종', '2종', '1종+SCR', '엔진교체', '삼원촉매'], dtype=object)

# 21\. 요소수 정보

In [68]:
name = 'TB_ERP_UREAWATER'
file_name = f'ODS_{name}.csv'
ureawater = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
ureawater.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ERP_REG_NO    80 non-null     int64 
 1   CAR_REG_NO    80 non-null     object
 2   CAR_SER_NO    80 non-null     object
 3   REC_DATE      80 non-null     int64 
 4   REC_NM        80 non-null     object
 5   LAS_CHG_DATE  80 non-null     object
 6   LAS_CHG_NM    80 non-null     object
dtypes: int64(2), object(5)
memory usage: 4.5+ KB


In [69]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
ureawater.columns = [col_dict[x] for x in ureawater.columns]
ureawater.columns

Index(['저감등록번호', '차량번호', '차대번호', '미정', '미정', '미정', '미정'], dtype='object')

In [70]:
ureawater1 = ureawater[['저감등록번호', '차량번호', '차대번호']]

In [71]:
ureawater1.isnull().sum()

저감등록번호    0
차량번호      0
차대번호      0
dtype: int64

In [72]:
col = pd.DataFrame(col_dict, index=['내용']).T.reset_index()
col.columns = ['컬럼명', '내용']
nul = ureawater1.isnull().sum().reset_index()
nul.columns = ['내용', '결측 값']
col2 = col.merge(nul, on='내용', how='left')
col2

,컬럼명,내용,결측 값
0,CAR_SER_NO,차대번호,0.00
1,ERP_REG_NO,저감등록번호,0.00
2,CAR_REG_NO,차량번호,0.00
3,REC_DATE,미정,NaN
4,REC_NM,미정,NaN
5,LAS_CHG_DATE,미정,NaN
6,LAS_CHG_NM,미정,NaN


# 22\. 요소수 상세정보

In [73]:
name = 'TB_ERP_UREAWATER_DTL'
file_name = f'ODS_{name}.csv'
ureawater_dtl = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
ureawater_dtl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ERP_REG_NO              155 non-null    int64  
 1   UREAWATER_CHK_STS       155 non-null    object 
 2   UREAWATER_LITER         155 non-null    int64  
 3   REMAIN_UREAWATER_LITER  155 non-null    int64  
 4   UREAWATER_MONEY         155 non-null    int64  
 5   REMAIN_UREAWATER_MONEY  155 non-null    int64  
 6   UREAWATER_COUNT         155 non-null    int64  
 7   UREAWATER_DATE          155 non-null    int64  
 8   UREAWATER_NM            155 non-null    object 
 9   LAS_CHG_DATE            155 non-null    object 
 10  LAS_CHG_NM              155 non-null    object 
 11  UREAWATER_FINAL_DATE    125 non-null    float64
 12  UREAWATER_FINAL_NM      125 non-null    object 
 13  RETURN_REASON           30 non-null     object 
dtypes: float64(1), int64(7), object(6)
memory 

In [74]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
ureawater_dtl.columns = [col_dict[x] for x in ureawater_dtl.columns]
ureawater_dtl.columns

Index(['저감등록번호', '요소수보충진행상태', '요소수보충리터', '요소수보충잔여리터', '요소수보충비용', '요소수보충잔여비용',
       '요소수보충횟수', '요소수보충일자', '미정', '미정', '미정', '미정', '미정', '미정'],
      dtype='object')

In [75]:
ureawater_dtl1 = ureawater_dtl[['저감등록번호', '요소수보충진행상태', '요소수보충리터', '요소수보충잔여리터', '요소수보충비용', '요소수보충잔여비용', '요소수보충횟수', '요소수보충일자']]

In [76]:
ureawater_dtl1.isnull().sum()

저감등록번호       0
요소수보충진행상태    0
요소수보충리터      0
요소수보충잔여리터    0
요소수보충비용      0
요소수보충잔여비용    0
요소수보충횟수      0
요소수보충일자      0
dtype: int64

In [77]:
col = pd.DataFrame(col_dict, index=['내용']).T.reset_index()
col.columns = ['컬럼명', '내용']
nul = ureawater_dtl1.isnull().sum().reset_index()
nul.columns = ['내용', '결측 값']
col2 = col.merge(nul, on='내용', how='left')
col2

,컬럼명,내용,결측 값
0,ERP_REG_NO,저감등록번호,0.00
1,REMAIN_UREAWATER_MONEY,요소수보충잔여비용,0.00
2,UREAWATER_CHK_STS,요소수보충진행상태,0.00
3,UREAWATER_DATE,요소수보충일자,0.00
4,REMAIN_UREAWATER_LITER,요소수보충잔여리터,0.00
5,UREAWATER_MONEY,요소수보충비용,0.00
6,UREAWATER_LITER,요소수보충리터,0.00
7,UREAWATER_COUNT,요소수보충횟수,0.00
8,UREAWATER_NM,미정,NaN
9,LAS_CHG_DATE,미정,NaN


# 23\. 전문정비

In [78]:
# about 34s
name = 'TB_ERR_RES'
file_name = f'ODS_{name}.csv'
res = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54617 entries, 0 to 54616
Data columns (total 42 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   REPAIR_ORG_NM       54617 non-null  object 
 1   CAR_REG_NO          54617 non-null  object 
 2   CAR_SER_NO          51051 non-null  object 
 3   REPAIR_DATE         54617 non-null  int64  
 4   REPAIR_REG_NO       54617 non-null  int64  
 5   RE_SMO_CHK_REG_NO   54617 non-null  object 
 6   CAR_NM              54615 non-null  object 
 7   CAR_YEAR            54615 non-null  float64
 8   CAR_OWNER_NM        54615 non-null  object 
 9   OFFICE_NM           25670 non-null  object 
 10  REPAIR_NM           54617 non-null  object 
 11  CAR_FUEL_NM         54615 non-null  object 
 12  FIRST_REG_DATE      25669 non-null  object 
 13  CAR_MILE            54617 non-null  float64
 14  REPAIR_COST         54617 non-null  object 
 15  WAR_REPAIR_YN       54617 non-null  object 
 16  RE_I

In [79]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
res.columns = [col_dict[x] for x in res.columns]
res.columns

Index(['전문정비업소명', '차량등록번호', '차대번호', '전문정비일', '전문정비접수번호', '정밀재검사접수번호', '미정',
       '미정', '미정', '미정', '미정', '미정', '미정', '주행거리', '미정', '미정', '미정', '미정',
       '미정', '미정', '부하일산화탄소측정치', '부하탄화수소측정치', '부하질소산화물측정치', '부하매연측정치',
       '부하토크측정치', '무부하일산화탄소', '무부하탄화수소', '무부하매연측정치', '무부하공기과잉률', '미정', '미정',
       '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정'],
      dtype='object')

In [80]:
res = res['전문정비일']

In [81]:
res.describe()

count      54617.00
mean    20083896.95
std        16435.80
min     20060727.00
25%     20070801.00
50%     20080507.00
75%     20091109.00
max     20121010.00
Name: 전문정비일, dtype: float64

In [82]:
res.head()

0    20070516
1    20070516
2    20070516
3    20070516
4    20070516
Name: 전문정비일, dtype: int64

In [83]:
res.isnull().sum()

0

In [85]:
for k, v in col_dict.items():
    if v == '전문정비일':
        print(k)

REPAIR_DATE


# 24\. 전문정비상세

In [86]:
name = 'TB_ERR_RES_DET'
file_name = f'ODS_{name}.csv'
res_det = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
res_det.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106818 entries, 0 to 106817
Data columns (total 24 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   REPAIR_ORG_NM      106818 non-null  object 
 1   CAR_REG_NO         106818 non-null  object 
 2   CAR_SER_NO         94670 non-null   object 
 3   REPAIR_DATE        106818 non-null  int64  
 4   REPAIR_REG_NO      106818 non-null  int64  
 5   RE_SMO_CHK_REG_NO  0 non-null       float64
 6   REPAIR_PART_CD     106817 non-null  object 
 7   REPAIR_CONT_CD     106817 non-null  object 
 8   REPAIR_PART1_YN    106818 non-null  object 
 9   REPAIR_PART2_YN    106818 non-null  object 
 10  REPAIR_PART3_YN    106818 non-null  object 
 11  REPAIR_PART4_YN    106818 non-null  object 
 12  REC_DATE           0 non-null       float64
 13  REC_NM             0 non-null       float64
 14  LAS_CHG_DATE       0 non-null       float64
 15  LAS_CHG_NM         0 non-null       float64
 16  ET

In [87]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
res_det.columns = [col_dict[x] for x in res_det.columns]
res_det.columns

Index(['전문정비업소명', '차량번호', '차대번호', '전문정비일', '전문정비접수번호', '정밀재검사접수번호', '정비부품코드',
       '정비내용코드', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정',
       '미정', '미정', '미정', '미정', '미정', '미정'],
      dtype='object')

In [88]:
res_det1 = res_det[['전문정비업소명', '차량번호', '차대번호', '전문정비일', '전문정비접수번호', '정밀재검사접수번호', '정비부품코드', '정비내용코드']]

In [89]:
res_det1.isnull().sum()

전문정비업소명           0
차량번호              0
차대번호          12148
전문정비일             0
전문정비접수번호          0
정밀재검사접수번호    106818
정비부품코드            1
정비내용코드            1
dtype: int64

In [90]:
col = pd.DataFrame(col_dict, index=['내용']).T.reset_index()
col.columns = ['컬럼명', '내용']
nul = res_det1.isnull().sum().reset_index()
nul.columns = ['내용', '결측 값']
col2 = col.merge(nul, on='내용', how='left')
col2

,컬럼명,내용,결측 값
0,CAR_REG_NO,차량번호,0.00
1,CAR_SER_NO,차대번호,12148.00
2,RE_SMO_CHK_REG_NO,정밀재검사접수번호,106818.00
3,REPAIR_REG_NO,전문정비접수번호,0.00
4,REPAIR_DATE,전문정비일,0.00
5,REPAIR_ORG_NM,전문정비업소명,0.00
6,REPAIR_PART_CD,정비부품코드,1.00
7,REPAIR_CONT_CD,정비내용코드,1.00
8,REPAIR_PART1_YN,미정,NaN
9,REPAIR_PART2_YN,미정,NaN


# 25\. 정기검사 검사정보

In [39]:
# about 3m 27s
name = 'TB_JGT_HIS'
file_name = f'ODS_{name}.csv'
his = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
his.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22006426 entries, 0 to 22006425
Data columns (total 63 columns):
 #   Column             Dtype  
---  ------             -----  
 0   JG_ORG_NM          object 
 1   JG_CHK_DATE        int64  
 2   JG_CHK_TYPE        object 
 3   JEBSU_NO           object 
 4   TO_JEBSU_NO        object 
 5   CAR_REG_NO         object 
 6   CAR_SER_NO         object 
 7   CAR_TYPE           object 
 8   CAR_NM             object 
 9   CAR_TYPE_NM        object 
 10  USAGE              object 
 11  CAR_YEAR           int64  
 12  CAR_FUEL_NM        object 
 13  CAR_MILE           object 
 14  CAR_LENGTH         float64
 15  CAR_WIDTH          float64
 16  CAR_HEIGHT         float64
 17  ENGINE_TYPE        object 
 18  ENGINE_CAPACITY    float64
 19  ENGINE_POWER       object 
 20  CAR_WEIGHT         float64
 21  TOT_WEIGHT         float64
 22  CAR_PASSENGER      float64
 23  CONT               float64
 24  FIRST_REG_DATE     int64  
 25  MAKE_DATE       

In [40]:
his['EXP_DATE'].isnull().sum()

3

In [5]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
his.columns = [col_dict[x] for x in his.columns]
his.columns

Index(['검사소명', '정기검사일자', '검사종류', '검사접수번호', '미정', '차량번호', '차대번호', '미정', '차명',
       '차종', '차량용도', '차량연식', '사용연료', '주행거리', '미정', '미정', '미정', '엔진형식', '미정',
       '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '검사방법',
       '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2',
       '무부하매연판정2', '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4',
       '무부하매연허용치4', '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5',
       '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6', '이산화탄소값', '최대출력허용치', '최대출력측정값',
       '산소값', '미정', '미정', '정기검사판정', '재검사기간', '미정', '미정', '미정', '미정', '검사판정시각',
       '미정', '미정'],
      dtype='object')

In [6]:
his['사용연료'].unique()

array(['경유', '휘발유(무연)', '엘피지', '휘발유', 'CNG', '하이브리드(휘발유+전기)',
       '하이브리드(LPG+전기)', '전기', nan, '기타연료', '하이브리드(CNG+전기)', 'HE(수소전기)',
       '하이브리드(경유+전기)', '휘발유(유연)', 'LNG', '태양열'], dtype=object)

In [7]:
his.isnull().sum()

검사소명            0
정기검사일자          0
검사종류            0
검사접수번호          0
미정              0
           ...   
미정        3344722
미정        3216631
검사판정시각          0
미정              0
미정             62
Length: 63, dtype: int64

In [11]:
his['차대번호'].isnull().sum()

0

In [12]:
his.isnull().sum().reset_index()

,index,0
0,검사소명,0
1,정기검사일자,0
2,검사종류,0
3,검사접수번호,0
4,미정,0
...,...,...
58,미정,3344722
59,미정,3216631
60,검사판정시각,0
61,미정,0


In [8]:
# about 3m 27s
name = '[ODS]정기검사(2022.12.28)'
file_name = f'{name}.csv'
ins = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
ins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22006426 entries, 0 to 22006425
Data columns (total 39 columns):
 #   Column     Dtype  
---  ------     -----  
 0   사용연료       object 
 1   주행거리       object 
 2   차명         object 
 3   차량번호       object 
 4   차대번호       object 
 5   차종         object 
 6   차량연식       int64  
 7   엔진형식       object 
 8   검사방법       object 
 9   검사일자       int64  
 10  검사종류       object 
 11  검사판정시각     object 
 12  검사판정       object 
 13  산소값        float64
 14  이산화탄소값     float64
 15  무부하매연판정1   object 
 16  무부하매연판정2   object 
 17  무부하매연판정3   object 
 18  무부하매연판정4   object 
 19  무부하매연판정5   object 
 20  무부하매연판정6   object 
 21  무부하매연허용치1  float64
 22  무부하매연허용치2  float64
 23  무부하매연허용치3  float64
 24  무부하매연허용치4  float64
 25  무부하매연허용치5  float64
 26  무부하매연허용치6  float64
 27  무부하매연측정치1  float64
 28  무부하매연측정치2  float64
 29  무부하매연측정치3  float64
 30  무부하매연측정치4  float64
 31  무부하매연측정치5  float64
 32  무부하매연측정치6  float64
 33  차량용도       object 
 34  최대출력허용치    float

In [9]:
ins.columns

Index(['사용연료', '주행거리', '차명', '차량번호', '차대번호', '차종', '차량연식', '엔진형식', '검사방법',
       '검사일자', '검사종류', '검사판정시각', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1',
       '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1',
       '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5',
       '무부하매연측정치6', '차량용도', '최대출력허용치', '최대출력측정값', '검사접수번호', '검사소명', '재검사기간'],
      dtype='object')

In [10]:
ins.isnull().sum()

사용연료           166418
주행거리            36446
차명                  0
차량번호                0
차대번호                0
차종                  0
차량연식                0
엔진형식            60820
검사방법                0
검사일자                0
검사종류                0
검사판정시각              0
검사판정                0
산소값          15533038
이산화탄소값       14435107
무부하매연판정1      1543914
무부하매연판정2     14375306
무부하매연판정3     14375352
무부하매연판정4     15971432
무부하매연판정5     15971432
무부하매연판정6     15971442
무부하매연허용치1     1543914
무부하매연허용치2    14375306
무부하매연허용치3    14375352
무부하매연허용치4    15971432
무부하매연허용치5    15971432
무부하매연허용치6    15971442
무부하매연측정치1     1543914
무부하매연측정치2    14375306
무부하매연측정치3    14375306
무부하매연측정치4    15971432
무부하매연측정치5    15971432
무부하매연측정치6    15971432
차량용도                0
최대출력허용치      22006426
최대출력측정값      22006426
검사접수번호              0
검사소명                0
재검사기간               0
dtype: int64

## 사용연료

In [ ]:
ins['사용연료'].unique()

array(['경유', '휘발유(무연)', '엘피지', '휘발유', 'CNG', '하이브리드(휘발유+전기)',
       '하이브리드(LPG+전기)', '전기', nan, '기타연료', '하이브리드(CNG+전기)', 'HE(수소전기)',
       '하이브리드(경유+전기)', '휘발유(유연)', 'LNG', '태양열'], dtype=object)

In [ ]:
ins.loc[ins['사용연료'].isnull() == True, ['사용연료', '차명']].head()

,사용연료,차명
399,NaN,한특20피트컨테이너샤시
414,NaN,인정20톤로우베드트레일러
755,NaN,한특40피트컨테이너샤시
1109,NaN,인정32㎥벌크트레일러
1175,NaN,JY카고트레일러


In [ ]:
ins.loc[ins['사용연료'].isnull() == True, ['사용연료', '차명']].iloc[50:60]

,사용연료,차명
16080,NaN,한특15M슬라이드 사이드티핑 평판트레일러
16175,NaN,두성40피트평판트레일러
16324,NaN,대한평판트레일러
16390,NaN,케스보러 콤비트레일러
16395,NaN,현대20/40피트콤보샤시
16439,NaN,한특15M슬라이드 사이드티핑 평판트레일러
16780,NaN,미래40피트콤비네이션콘테이너샤시
16824,NaN,우주슬라이딩평판트레일러
16969,NaN,대흥가변식저상트레일러
17356,NaN,현대차량 수송트레일러


## 차명

In [ ]:
ins['차명'].isnull().sum()

0

## 차량번호

In [ ]:
ins['차량번호'].isnull().sum()

0

In [ ]:
ins['차량번호'].unique()

array(['86마9529', '10주0782', '28호4612', ..., '22부1779', '48가4902',
       '28오6600'], dtype=object)

In [ ]:
p1 = re.compile('[가-힣]{2}[0-9]{1,2}[가-힣]{1}[0-9]{4}')
p2 = re.compile('[0-9]{2,3}[가-힣]{1}[0-9]{4}')
wrong_vhrno_list = []
for one in tqdm(ins['차량번호']):
    if not p1.search(str(one)) and not p2.search(str(one)):
        wrong_vhrno_list.append(one)
len(wrong_vhrno_list)

100%|██████████| 22006426/22006426 [00:27<00:00, 791943.15it/s]


312

In [ ]:
wrong_vhrno_list[:5]

['#4조2591', '#5러4748', '#8나1304', '#8나1304', '#5러4748']

In [ ]:
wrong_vhrno_list[-5:]

['#0두0223', '#0두0223', '#0두0223', '#0두0223', '#0두0223']

## 차대번호

In [ ]:
vin_temp = ins['차대번호'].str.len()
set(vin_temp.unique())

{9, 10, 11, 12, 13, 14, 15, 16, 17}

In [ ]:
count = 0
for one in set(vin_temp.unique()):
    temp = ins.loc[ins['차대번호'].str.len() == one, '차대번호']
    count += temp.shape[0]
    print(f'차대번호 길이 : {one}')
    print(temp.shape)
    display(temp.head())
    print()
count

차대번호 길이 : 9
(10,)


1801869     CN67C4394
5139662     CN67C4394
7746532     410133836
9076446     CN67C4394
11788799    CN67C4394
Name: 차대번호, dtype: object


차대번호 길이 : 10
(4,)


431260      PN71C02663
5403985     0384200247
5406660     0384200247
16505232    0384200247
Name: 차대번호, dtype: object


차대번호 길이 : 11
(2361,)


...


차대번호 길이 : 16
(245,)


204150    KLYKA84D7C289437
346928    KMHDC51DP9163835
358233    11-94-342-10012가
358234    11-94-342-10012가
374613    KMHBU51D9U891793
Name: 차대번호, dtype: object


차대번호 길이 : 17
(21984808,)


0    KNCWJX76AGK038342
1    KMHFF41CBDA233234
2    KNAGN418BFA579667
3    KNFSTZ76AJK245830
4    KMFWBX7KBJU977503
Name: 차대번호, dtype: object

22006426

In [ ]:
len_17_wrong_text_idx = []
p = re.compile('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]')
for i, one in enumerate(ins['차대번호']):
    if len(str(one)) == 17 and p.search(str(one)):
        len_17_wrong_text_idx.append(i)
len(len_17_wrong_text_idx)

0

In [ ]:
ins.loc[len_17_wrong_text_idx, '차대번호'].head()

Series([], Name: 차대번호, dtype: object)

## 차종

In [ ]:
ins['차종'].isnull().sum()

0

In [ ]:
ins['차종'].unique()

array(['화물차일반형중형', '승용차일반형대형', '승용차일반형중형', '화물차특수용도형소형', '화물차밴형소형',
       '화물차일반형소형', '승용차다목적형대형', '승용차일반형경형', '승합차일반형중형', '특수차특수작업형소형',
       '승용차기타형중형', '화물차일반형대형', '화물차특수용도형대형', '승용차기타형대형', '승용차다목적형중형',
       '승용차일반형소형', '승합차일반형대형', '화물차밴형경형', '화물차일반형경형', '특수차구난형소형',
       '특수차견인형대형', '화물차밴형중형', '화물차덤프형대형', '화물차덤프형소형', '특수차특수작업형중형',
       '화물차특수용도형중형', '화물차덤프형중형', '승합차특수형중형', '승합차일반형경형', '화물차피견인형적재함형',
       '화물차탱크로리형대형', '화물차피견인형저상형', '승용차겸화물형경형', '화물차특수용도형경형',
       '화물차탱크로리형중형', '특수차특수작업형대형', '화물차피견인형샷시형', '승용차다목적형소형', '승용차겸화물형대형',
       '특수차구난형대형', '특수차구난형중형', '화물차피견인형고상형', '승합차일반형소형', '특수차견인형소형',
       '화물차탱크로리형경형', '승합차특수형대형', '화물차피견인형기타형', '승합차특수형소형', '화물차탱크로리형소형',
       '승합차특수형경형', '특수차특수작업형경형', '화물차밴형대형', '특수차견인형중형', '승용차기타형경형',
       '승용차기타형소형', '승용차겸화물형소형', '화물차덤프형경형', '승용차겸화물형중형', '특수차특수용도형중형',
       '특수차특수용도형소형', '특수차특수용도형대형', '특수차특수용도형경형', '승용차다목적형경형'],
      dtype=object)

In [ ]:
# '승용차일반형경형',
# '승용차일반형소형',
# '승용차일반형중형',
# '승용차일반형대형',

# '승용차겸화물형경형',
# '승용차겸화물형소형',
# '승용차겸화물형중형',
# '승용차겸화물형대형',

# '승용차다목적형경형',
# '승용차다목적형소형',
# '승용차다목적형중형',
# '승용차다목적형대형',

# '승용차기타형경형',
# '승용차기타형소형',
# '승용차기타형중형',
# '승용차기타형대형',

In [ ]:
# '승합차일반형경형',
# '승합차일반형소형',
# '승합차일반형중형',
# '승합차일반형대형',

# '승합차특수형경형',
# '승합차특수형소형',
# '승합차특수형중형',
# '승합차특수형대형',

In [ ]:
# '화물차일반형경형',
# '화물차일반형소형',
# '화물차일반형중형',
# '화물차일반형대형',

# '화물차밴형경형',
# '화물차밴형소형',
# '화물차밴형중형',
# '화물차밴형대형',

# '화물차덤프형경형',
# '화물차덤프형소형',
# '화물차덤프형중형',
# '화물차덤프형대형',

# '화물차탱크로리형경형',
# '화물차탱크로리형소형',
# '화물차탱크로리형중형',
# '화물차탱크로리형대형',

# '화물차피견인형저상형',
# '화물차피견인형고상형',
# '화물차피견인형샷시형',
# '화물차피견인형적재함형',
# '화물차피견인형기타형',

# '화물차특수용도형경형',
# '화물차특수용도형소형',
# '화물차특수용도형중형',
# '화물차특수용도형대형',

In [ ]:
# '특수차특수작업형경형',
# '특수차특수작업형소형',
# '특수차특수작업형중형',
# '특수차특수작업형대형',

# '특수차구난형소형',
# '특수차구난형중형',
# '특수차구난형대형',

# '특수차견인형소형',
# '특수차견인형중형',
# '특수차견인형대형',

# '특수차특수용도형경형',
# '특수차특수용도형소형',
# '특수차특수용도형중형',
# '특수차특수용도형대형',

## 차량연식

In [ ]:
ins['차량연식'].isnull().sum()

0

In [ ]:
ins['차량연식'].describe()

count   22006426.00
mean        2013.37
std            5.59
min         1904.00
25%         2010.00
50%         2015.00
75%         2018.00
max         2023.00
Name: 차량연식, dtype: float64

In [ ]:
ins['차량연식'].head()

0    2016
1    2013
2    2015
3    2018
4    2018
Name: 차량연식, dtype: int64

In [ ]:
count = 0
for one in ins['차량연식']:
    if len(str(one)) != 4:
        count += 1
count

0

## 검사방법

In [ ]:
ins['검사방법'].isnull().sum()

0

In [ ]:
ins['검사방법'].unique()

array(['무부하검사(급가속)', '무부하검사(TSI)', '무부하검사(정지가동)'], dtype=object)

## 엔진형식

In [ ]:
ins['엔진형식'].isnull().sum()

60820

In [ ]:
ins.loc[ins['엔진형식'].isnull() == True, ['엔진형식', '차명']].head()

,엔진형식,차명
399,NaN,한특20피트컨테이너샤시
414,NaN,인정20톤로우베드트레일러
755,NaN,한특40피트컨테이너샤시
1569,NaN,케스보러 콤비트레일러
2586,NaN,한특40피트콤비네이션샤시


## 정기검사일자

In [ ]:
ins.columns

Index(['사용연료', '주행거리', '차명', '차량번호', '차대번호', '차종', '차량연식', '엔진형식', '검사방법',
       '검사일자', '검사종류', '검사판정시각', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1',
       '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1',
       '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5',
       '무부하매연측정치6', '차량용도', '최대출력허용치', '최대출력측정값', '검사접수번호', '검사소명', '재검사기간'],
      dtype='object')

In [ ]:
ins['검사일자'].isnull().sum()

0

In [ ]:
ins['검사일자'].unique()

array([20190102, 20190103, 20190104, ..., 20221230, 20221231, 20230102],
      dtype=int64)

In [ ]:
count = 0
for one in ins['검사일자']:
    if len(str(one)) != 8:
        count += 1
count

0

## 검사종류

In [ ]:
ins['검사종류'].isnull().sum()

0

In [ ]:
ins['검사종류'].unique()

array(['정기검사', '재검사(1부재검사)', '정기(경과)검사', '재검사(2부재검사)'], dtype=object)

# 26\. 정기검사 검사소정보

In [91]:
name = 'TB_JGT_VIO'
file_name = f'ODS_{name}.csv'
jgt_vio = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
jgt_vio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1961 entries, 0 to 1960
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   VIO_CD    1961 non-null   object
 1   VIO_NM    1961 non-null   object
 2   VIO_TYPE  1961 non-null   int64 
 3   SIDO_NM   1961 non-null   object
dtypes: int64(1), object(3)
memory usage: 61.4+ KB


In [92]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
jgt_vio.columns = [col_dict[x] for x in jgt_vio.columns]
jgt_vio.columns

Index(['검사소코드', '검사소명', '검사소구분', '시도명'], dtype='object')

In [93]:
jgt_vio1 = jgt_vio[['검사소코드', '검사소명', '검사소구분', '시도명']]

In [94]:
jgt_vio1.isnull().sum()

검사소코드    0
검사소명     0
검사소구분    0
시도명      0
dtype: int64

In [95]:
col = pd.DataFrame(col_dict, index=['내용']).T.reset_index()
col.columns = ['컬럼명', '내용']
nul = jgt_vio1.isnull().sum().reset_index()
nul.columns = ['내용', '결측 값']
col2 = col.merge(nul, on='내용', how='left')
col2

,컬럼명,내용,결측 값
0,SIDO_NM,시도명,0
1,VIO_TYPE,검사소구분,0
2,VIO_CD,검사소코드,0
3,VIO_NM,검사소명,0


# 27\. 자동차 검사소정보

In [96]:
name = 'TB_MECAR_VIO'
file_name = f'ODS_{name}.csv'
mecar_vio = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
mecar_vio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1045 entries, 0 to 1044
Data columns (total 28 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   VIO_CD      1045 non-null   object 
 1   VIO_NM      1045 non-null   object 
 2   VIO_TYPE    1045 non-null   int64  
 3   RECV_DATE   134 non-null    float64
 4   CONF_DATE   134 non-null    float64
 5   BIZ_YN      1045 non-null   object 
 6   J_YN        1044 non-null   object 
 7   G_YN        219 non-null    object 
 8   S_YN        353 non-null    object 
 9   H_YN        387 non-null    object 
 10  SIDO_CD     1045 non-null   int64  
 11  SIGUNGU_CD  1043 non-null   float64
 12  SIDO_NM     1045 non-null   object 
 13  SIGUNGU_NM  1044 non-null   object 
 14  TEL_NO      435 non-null    object 
 15  POST_NO     428 non-null    object 
 16  ADDR1       1045 non-null   object 
 17  ADDR2       217 non-null    object 
 18  X_COORD     1045 non-null   int64  
 19  Y_COORD     1045 non-null  

In [97]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
mecar_vio.columns = [col_dict[x] for x in mecar_vio.columns]
mecar_vio.columns

Index(['검사소코드', '검사소명', '검사소구분', '미정', '미정', '미정', '미정', '미정', '미정', '미정',
       '미정', '미정', '시도명', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정',
       '미정', '미정', '미정', '미정', '미정', '미정'],
      dtype='object')

In [98]:
mecar_vio1 = mecar_vio[['검사소코드', '검사소명', '검사소구분', '시도명']]

In [99]:
mecar_vio1.isnull().sum()

검사소코드    0
검사소명     0
검사소구분    0
시도명      0
dtype: int64

In [100]:
col = pd.DataFrame(col_dict, index=['내용']).T.reset_index()
col.columns = ['컬럼명', '내용']
nul = mecar_vio1.isnull().sum().reset_index()
nul.columns = ['내용', '결측 값']
col2 = col.merge(nul, on='내용', how='left')
col2

,컬럼명,내용,결측 값
0,VIO_NM,검사소명,0.00
1,VIO_CD,검사소코드,0.00
2,SIDO_NM,시도명,0.00
3,VIO_TYPE,검사소구분,0.00
4,RECV_DATE,미정,NaN
5,CONF_DATE,미정,NaN
6,BIZ_YN,미정,NaN
7,J_YN,미정,NaN
8,G_YN,미정,NaN
9,S_YN,미정,NaN


# 28\. RSD 수시점검

In [115]:
# about 4m 5s
name = 'TB_RSD_FINAL'
file_name = f'ODS_{name}.csv'
rsd = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
rsd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13318300 entries, 0 to 13318299
Data columns (total 74 columns):
 #   Column             Dtype  
---  ------             -----  
 0   IDX                int64  
 1   UNIT               object 
 2   SITECODE           object 
 3   CAR_REG_NO         object 
 4   CAR_SER_NO         object 
 5   RSD_DATE           int64  
 6   RSD_TIME           object 
 7   CO                 float64
 8   PPMHC              float64
 9   PPMNO              float64
 10  CAR_NM             object 
 11  USAGE              object 
 12  CAR_TYPE_NM        object 
 13  CAR_YEAR           int64  
 14  CAR_FUEL_NM        object 
 15  CAR_WEIGHT         int64  
 16  TOT_WEIGHT         int64  
 17  ENGINE_CAPACITY    float64
 18  ENGINE_POWER       object 
 19  FIRST_REG_DATE     int64  
 20  MAKE_REG_DATE      object 
 21  SMO_CHK_EXP_DATE   int64  
 22  SPEED              float64
 23  ACCEL              float64
 24  VSP                float64
 25  UVSMOKE         

In [116]:
file_name = f'[정제]{name}.xlsx'
col_df = pd.read_excel(os.path.join(col_fold, file_name), sheet_name='ODS_col')
col_dict = {a:b for a,b in col_df[['컬럼', '내용']].values}
rsd.columns = [col_dict[x] for x in rsd.columns]
rsd.columns

Index(['미정', '장비코드', '측정위치코드', '차량번호', '차대번호', '측정일자', '측정시각', '일산화탄소측정치',
       'HC측정치', 'NOX측정치', '미정', '미정', '미정', '모델연도', '차량연료', '공차중량', '미정', '미정',
       '미정', '미정', '미정', '미정', '차량현재속도', '엑셀', 'VSP', '매연측정치', '미정', 'IRSMOKE',
       '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정', '미정',
       '미정', '미정', '미정', '미정', '미정', '미정', '미정', '용도명칭', '미정', '미정', '미정',
       '미정', '미정', '미정', '미정', 'PPMHCHEX', '미정', '미정', '미정', '미정', '미정', '미정',
       '미정', '미정', '미정', '미정', '법정동 시도명', '법정동시군구명', '과다구분', '엔진형식', '미정',
       '최대 불투명도(매연측정값)', '측정기구분', 'PPM_NO2'],
      dtype='object')

In [117]:
rsd = rsd[['차량연료', '차량번호', '차대번호', '공차중량', '모델연도', '측정일자']]

In [118]:
rsd.isnull().sum()

차량연료    0
차량번호    0
차대번호    0
공차중량    0
모델연도    0
측정일자    0
dtype: int64

## 차량연료

In [119]:
rsd['차량연료'].unique()

array(['엘피지', '경유', '휘발유(무연)', '휘발유', 'CNG', '하이브리드(LPG+전기)',
       '하이브리드(휘발유+전기)', '휘발유(유연)', 'LNG', '하이브리드(CNG+전기)'], dtype=object)

In [120]:
len(rsd['차량연료'].unique())

10

## 차량번호

In [121]:
rsd['차량번호'].isnull().sum()

0

In [122]:
rsd['차량번호'].unique()

array(['인천33구3715', '인천33구6342', '인천33노3293', ..., '96거6566', '96고7246',
       '96더9690'], dtype=object)

In [123]:
p1 = re.compile('[가-힣]{2}[0-9]{1,2}[가-힣]{1}[0-9]{4}')
p2 = re.compile('[0-9]{2,3}[가-힣]{1}[0-9]{4}')
wrong_vhrno_list = []
for one in tqdm(rsd['차량번호']):
    if not p1.search(str(one)) and not p2.search(str(one)):
        wrong_vhrno_list.append(one)
len(wrong_vhrno_list)

100%|██████████| 13318300/13318300 [00:17<00:00, 759725.24it/s]


0

In [124]:
wrong_vhrno_list[:5]

[]

In [125]:
wrong_vhrno_list[-5:]

[]

## 차대번호

In [126]:
vin_temp = rsd['차대번호'].str.len()
set(vin_temp.unique())

{11, 12, 13, 14, 15, 16, 17}

In [127]:
count = 0
for one in set(vin_temp.unique()):
    temp = rsd.loc[rsd['차대번호'].str.len() == one, '차대번호']
    count += temp.shape[0]
    print(f'차대번호 길이 : {one}')
    print(temp.shape)
    display(temp.head())
    print()
count

차대번호 길이 : 11
(40,)


52833     01901322053
63676     01901321653
215588    01861321295
238416    01861321295
814190    WFY11275737
Name: 차대번호, dtype: object


차대번호 길이 : 12
(1646,)


9708     110011326311
10059    410234336364
10198    439934203900
13234    119211312391
14843    410234337409
Name: 차대번호, dtype: object


차대번호 길이 : 13
(36,)


...


차대번호 길이 : 16
(149,)


12771     KNMC4C2HM7069548
17500     KNAK518BBT954434
103316    KNAK518BBT954434
113795    KMJFD37B1K518060
183906    KNAFH22139A00852
Name: 차대번호, dtype: object


차대번호 길이 : 17
(13316394,)


0    KMHEN41MP2A794772
1    KMHNN81XP3U088490
2    KMJWWH7VPYU276950
3    KMHCG41GP1U298190
4    KMHSC81VP1U190763
Name: 차대번호, dtype: object

13318300

In [128]:
len_17_wrong_text_idx = []
p = re.compile('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]')
for i, one in enumerate(rsd['차대번호']):
    if len(str(one)) == 17 and p.search(str(one)):
        len_17_wrong_text_idx.append(i)
len(len_17_wrong_text_idx)

0

In [129]:
rsd.loc[len_17_wrong_text_idx, '차대번호'].head()

Series([], Name: 차대번호, dtype: object)

## 공차중량

In [130]:
rsd['공차중량'].head()

0    1435
1    2115
2    1835
3    1015
4    1645
Name: 공차중량, dtype: int64

In [131]:
rsd['공차중량'].describe()

count   13318300.00
mean        1704.95
std          764.29
min            0.00
25%         1445.00
50%         1640.00
75%         1905.00
max        35110.00
Name: 공차중량, dtype: float64

## 모델연도

In [132]:
rsd['모델연도'].head()

0    2002
1    2003
2    2000
3    2001
4    2001
Name: 모델연도, dtype: int64

In [133]:
rsd['모델연도'].describe()

count   13318300.00
mean        2012.87
std            5.07
min         1960.00
25%         2010.00
50%         2014.00
75%         2017.00
max         2023.00
Name: 모델연도, dtype: float64

## 측정일자

In [134]:
rsd['측정일자'].head()

0    20130304
1    20130304
2    20130304
3    20130304
4    20130304
Name: 측정일자, dtype: int64

In [135]:
rsd['측정일자'].describe()

count   13318300.00
mean    20182459.99
std        28027.56
min     20130304.00
25%     20160714.00
50%     20190302.00
75%     20210228.00
max     20221222.00
Name: 측정일자, dtype: float64

# 코드 마지막